In [ ]:
#@title 🔥 SHA-256 SAT PIPELINE + BITCOIN-LIKE PREIMAGE NAVIGATOR (OPTIMIZADO)
# Genera CNF, resuelve una vez y luego busca preimágenes tipo Bitcoin (leading zeros)
!pip install python-sat -q

import numpy as np
import math
import hashlib
import re
import time
from collections import defaultdict, deque
from pysat.solvers import Solver as PySolver

# ============================================================
# 1) CNFBuilder - SHA-256 en CNF (UNIVERSAL, SIN OBJETIVO)
# ============================================================

class CNFBuilder:
    def __init__(self):
        self.var_count = 0
        self.clauses = []
        self.var_map = {}

        self.K = [
            0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
            0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
            0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
            0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
            0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
            0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
            0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
            0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
        ]

        self.H_init = [
            0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
            0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
        ]

    def new_var(self, name=None):
        self.var_count += 1
        if name:
            self.var_map[name] = self.var_count
        return self.var_count

    def add_clause(self, literals):
        self.clauses.append(literals)

    def gate_xor(self, a, b):
        c = self.new_var()
        self.add_clause([-a, -b, -c])
        self.add_clause([a, b, -c])
        self.add_clause([a, -b, c])
        self.add_clause([-a, b, c])
        return c

    def gate_and(self, a, b):
        c = self.new_var()
        self.add_clause([-a, -b, c])
        self.add_clause([a, -c])
        self.add_clause([b, -c])
        return c

    def gate_not(self, a):
        c = self.new_var()
        self.add_clause([-a, -c])
        self.add_clause([a, c])
        return c

    def gate_maj(self, a, b, c):
        out = self.new_var()
        self.add_clause([-a, -b, out])
        self.add_clause([-a, -c, out])
        self.add_clause([-b, -c, out])
        self.add_clause([a, b, -out])
        self.add_clause([a, c, -out])
        self.add_clause([b, c, -out])
        return out

    def gate_ch(self, e, f, g):
        out = self.new_var()
        self.add_clause([-e, -f, out])
        self.add_clause([-e, f, -out])
        self.add_clause([e, -g, out])
        self.add_clause([e, g, -out])
        return out

    def const_word(self, val):
        bits = []
        for i in range(32):
            bit_val = (val >> i) & 1
            v = self.new_var()
            if bit_val:
                self.add_clause([v])
            else:
                self.add_clause([-v])
            bits.append(v)
        return bits  # LSB-first

    def xor_word(self, w1, w2):
        return [self.gate_xor(a, b) for a, b in zip(w1, w2)]

    def and_word(self, w1, w2):
        return [self.gate_and(a, b) for a, b in zip(w1, w2)]

    def not_word(self, w):
        return [self.gate_not(b) for b in w]

    def rot_right(self, w, n):
        return w[n:] + w[:n]

    def shift_right(self, w, n):
        zeros = []
        for _ in range(n):
            z = self.new_var()
            self.add_clause([-z])
            zeros.append(z)
        return w[n:] + zeros

    def add_word(self, w1, w2):
        result = []
        carry = None
        for i in range(32):
            a = w1[i]
            b = w2[i]
            if carry is None:
                s = self.gate_xor(a, b)
                c_out = self.gate_and(a, b)
            else:
                tmp = self.gate_xor(a, b)
                s = self.gate_xor(tmp, carry)
                ab = self.gate_and(a, b)
                cin_xor = self.gate_and(carry, tmp)
                c_out = self.new_var()
                self.add_clause([-ab, c_out])
                self.add_clause([-cin_xor, c_out])
                self.add_clause([ab, cin_xor, -c_out])
            result.append(s)
            carry = c_out
        return result

    def sigma0(self, w):
        r7 = self.rot_right(w, 7)
        r18 = self.rot_right(w, 18)
        s3 = self.shift_right(w, 3)
        tmp = self.xor_word(r7, r18)
        return self.xor_word(tmp, s3)

    def sigma1(self, w):
        r17 = self.rot_right(w, 17)
        r19 = self.rot_right(w, 19)
        s10 = self.shift_right(w, 10)
        tmp = self.xor_word(r17, r19)
        return self.xor_word(tmp, s10)

    def Sigma0(self, w):
        r2 = self.rot_right(w, 2)
        r13 = self.rot_right(w, 13)
        r22 = self.rot_right(w, 22)
        tmp = self.xor_word(r2, r13)
        return self.xor_word(tmp, r22)

    def Sigma1(self, w):
        r6 = self.rot_right(w, 6)
        r11 = self.rot_right(w, 11)
        r25 = self.rot_right(w, 25)
        tmp = self.xor_word(r6, r11)
        return self.xor_word(tmp, r25)

    def Maj_word(self, x, y, z):
        return [self.gate_maj(a, b, c) for a, b, c in zip(x, y, z)]

    def Ch_word(self, x, y, z):
        return [self.gate_ch(a, b, c) for a, b, c in zip(x, y, z)]

    def build_sha256(self):
        print("Generando variables de entrada (512 bits)...")
        M_block = []
        for i in range(16):
            word = []
            for j in range(32):
                v = self.new_var(f"msg_w{i}_b{j}")
                word.append(v)
            M_block.append(word)

        print("Expandiendo Message Schedule (W0..W63)...")
        W = [None] * 64
        for i in range(16):
            W[i] = M_block[i]

        for i in range(16, 64):
            if i % 5 == 0:
                print(f"  ...Ronda de expansión {i}/64")
            s1 = self.sigma1(W[i-2])
            w7 = W[i-7]
            s0 = self.sigma0(W[i-15])
            w16 = W[i-16]
            t1 = self.add_word(s1, w7)
            t2 = self.add_word(s0, w16)
            W[i] = self.add_word(t1, t2)

        print("Inicializando estado hash...")
        state = [self.const_word(h) for h in self.H_init]
        a, b, c, d, e, f, g, h = state

        print("Ejecutando Compresión (64 Rondas)...")
        for i in range(64):
            if i % 10 == 0:
                print(f"  ...Compresión ronda {i}/64")
            S1_e = self.Sigma1(e)
            ch_efg = self.Ch_word(e, f, g)
            ki = self.const_word(self.K[i])
            wi = W[i]

            sum1 = self.add_word(h, S1_e)
            sum2 = self.add_word(ch_efg, ki)
            sum3 = self.add_word(sum1, sum2)
            T1 = self.add_word(sum3, wi)

            S0_a = self.Sigma0(a)
            maj_abc = self.Maj_word(a, b, c)
            T2 = self.add_word(S0_a, maj_abc)

            h = g
            g = f
            f = e
            e = self.add_word(d, T1)
            d = c
            c = b
            b = a
            a = self.add_word(T1, T2)

        print("Suma final con estado inicial...")
        final_state_vars = [a, b, c, d, e, f, g, h]
        initial_state_vars = [self.const_word(hv) for hv in self.H_init]

        digest_bits = []
        for i in range(8):
            final_word = self.add_word(final_state_vars[i], initial_state_vars[i])
            digest_bits.extend(final_word)
            for bit_idx, var in enumerate(final_word):
                self.var_map[f"hash_w{i}_b{bit_idx}"] = var

        print(f"¡Hecho! Generadas {self.var_count} variables y {len(self.clauses)} cláusulas.")
        return digest_bits

    def save_dimacs(self, filename):
        print(f"Guardando en {filename}...")
        with open(filename, 'w') as f:
            f.write(f"p cnf {self.var_count} {len(self.clauses)}\n")
            f.write("c Mapeo Input: msg_wX_bY -> variables iniciales\n")
            for clause in self.clauses:
                f.write(" ".join(map(str, clause)) + " 0\n")
        with open(filename + ".map", "w") as f:
            for k, v in self.var_map.items():
                f.write(f"{k}:{v}\n")

# ============================================================
# 2) SATSolverQuaternion + guardar_resultado
# ============================================================

class SATSolverQuaternion:
    def __init__(self):
        self.num_vars = 0
        self.clauses = []
        self.var_pos = None
        self.var_neg = None
        self.var_clauses = None
        self.implications = None

    def parse_cnf_content(self, content, filename=""):
        self.clauses = []
        max_var = 0
        for line in content.split('\n'):
            line = line.strip()
            if not line or line[0] in 'cp%':
                if line.startswith('p cnf'):
                    parts = line.split()
                    if len(parts) >= 4:
                        self.num_vars = int(parts[2])
                continue
            clause = []
            for tok in line.split():
                try:
                    lit = int(tok)
                    if lit == 0:
                        if clause:
                            self.clauses.append(tuple(clause))
                            clause = []
                    else:
                        clause.append(lit)
                        max_var = max(max_var, abs(lit))
                except:
                    pass
            if clause:
                self.clauses.append(tuple(clause))
        self.num_vars = max(self.num_vars, max_var)
        self._build_structures()
        return len(self.clauses) > 0

    def _build_structures(self):
        n = self.num_vars
        self.var_pos = np.zeros(n + 1, dtype=np.float64)
        self.var_neg = np.zeros(n + 1, dtype=np.float64)
        self.var_clauses = [[] for _ in range(n + 1)]
        self.implications = {i: set() for i in range(-n, n + 1) if i != 0}

        for idx, clause in enumerate(self.clauses):
            clause_len = len(clause)
            peso = 1.0 / clause_len
            for lit in clause:
                var = abs(lit)
                if lit > 0:
                    self.var_pos[var] += peso
                else:
                    self.var_neg[var] += peso
                self.var_clauses[var].append(idx)
            if clause_len == 2:
                a, b = clause
                self.implications[-a].add(b)
                self.implications[-b].add(a)

    def contar_satisfechas(self, asignaciones):
        count = 0
        for clause in self.clauses:
            for lit in clause:
                var = abs(lit)
                val = asignaciones.get(var, True)
                if (lit > 0 and val) or (lit < 0 and not val):
                    count += 1
                    break
        return count

    def _heuristic_quaternion(self):
        n_vars = self.num_vars
        n_clauses = len(self.clauses)
        asignaciones = {}
        forzadas = set()

        for clause in self.clauses:
            if len(clause) == 1:
                lit = clause[0]
                var = abs(lit)
                asignaciones[var] = (lit > 0)
                forzadas.add(var)

        cambio = True
        pasadas = 0
        max_pasadas = int(np.log2(n_vars + 1)) + 2

        while cambio and pasadas < max_pasadas:
            cambio = False
            pasadas += 1
            for var in list(asignaciones.keys()):
                val = asignaciones[var]
                lit = var if val else -var
                if lit in self.implications:
                    for impl in self.implications[lit]:
                        impl_var = abs(impl)
                        impl_val = (impl > 0)
                        if impl_var not in asignaciones:
                            asignaciones[impl_var] = impl_val
                            forzadas.add(impl_var)
                            cambio = True

        class Quaternion:
            __slots__ = ['q']
            def __init__(self, w=0.0, x=0.0, y=0.0, z=0.0):
                self.q = np.array([w, x, y, z], dtype=np.float64)
            @property
            def w(self): return self.q[0]
            @property
            def x(self): return self.q[1]
            @property
            def y(self): return self.q[2]
            @property
            def z(self): return self.q[3]
            def __add__(self, other):
                r = Quaternion()
                r.q = self.q + other.q
                return r
            def norm(self):
                return np.sqrt(np.sum(self.q ** 2))
            def to_matrix(self):
                w, x, y, z = self.q
                return np.array([
                    [w, -x, -y, -z],
                    [x,  w, -z,  y],
                    [y,  z,  w, -x],
                    [z, -y,  x,  w]
                ], dtype=np.float64)

        vars_restantes = [v for v in range(1, n_vars + 1) if v not in asignaciones]
        q_global = Quaternion(0, 0, 0, 0)

        for var in vars_restantes:
            pos = self.var_pos[var]
            neg = self.var_neg[var]
            total = pos + neg
            if total < 1e-10:
                asignaciones[var] = True
                continue
            A = var
            B = 1 if pos >= neg else -1
            C = len(self.var_clauses[var])
            n = A - B + C
            if n == 0:
                n = 1
            a = (pos - neg) / (total + 1)
            b = total / (n_vars + 1)
            if b > 0:
                b_power = np.exp(np.log(b + 1) / max(abs(n), 1))
            else:
                b_power = 1.0
            q = Quaternion(
                w = a + b_power,
                x = B * pos / (total + 1),
                y = neg / (total + 1),
                z = np.log(abs(n) + 1) / np.log(n_vars + 2)
            )
            q_global = q_global + q

        n_coef = n_clauses / (n_vars + 1) + 1
        norm_g = q_global.norm()
        factor = np.log(norm_g + 1) / (n_coef + 1) if norm_g > 1e-10 else 1.0

        derivada = Quaternion(
            w = q_global.w * factor + q_global.x * 0.3,
            x = q_global.x * factor + q_global.y * 0.3,
            y = q_global.y * factor + q_global.z * 0.3,
            z = q_global.z * factor + q_global.w * 0.3
        )

        M = derivada.to_matrix()
        eigenvalues = np.linalg.eigvals(M)
        eigen_signs = np.sign(np.real(eigenvalues))
        valores_matriz = M.flatten()

        for var in vars_restantes:
            if var in asignaciones:
                continue
            pos = self.var_pos[var]
            neg = self.var_neg[var]
            total = pos + neg
            idx = (var - 1) % 16
            val_matriz = valores_matriz[idx]
            eigen_idx = (var - 1) % 4
            val_eigen = eigen_signs[eigen_idx]
            if total > 0:
                polaridad = (pos - neg) / total
            else:
                polaridad = 0
            score = (
                0.20 * np.sign(val_matriz) +
                0.15 * val_eigen +
                0.35 * np.sign(polaridad)
            )
            asignaciones[var] = (score >= 0)
        return asignaciones

    def _pysat_decidir(self):
        solver = PySolver(name='g3')
        for clause in self.clauses:
            solver.add_clause(list(clause))
        sat = solver.solve()
        if not sat:
            solver.delete()
            return "UNSAT", None
        modelo = solver.get_model()
        solver.delete()
        asignaciones = {}
        for lit in modelo:
            var = abs(lit)
            asignaciones[var] = (lit > 0)
        return "SAT", asignaciones

    def solve(self):
        n_vars = self.num_vars
        n_clauses = len(self.clauses)
        if n_clauses == 0:
            return "SAT", {i: True for i in range(1, n_vars + 1)}, 0, None
        print("  [1] Aproximación cuaterniónica O(log n)...")
        asig_heur = self._heuristic_quaternion()
        sat_heur = self.contar_satisfechas(asig_heur)
        pct_heur = sat_heur * 100.0 / n_clauses
        print(f"      Heurístico: {sat_heur}/{n_clauses} ({pct_heur:.2f}%)")
        if sat_heur == n_clauses:
            return "SAT", asig_heur, np.log(n_vars + n_clauses + 1), asig_heur
        print("  [2] Decisión exacta con PySAT (CDCL)...")
        resultado, asig_exact = self._pysat_decidir()
        return resultado, asig_heur, np.log(n_vars + n_clauses + 1), asig_exact

def guardar_resultado(filename, asig_final, num_vars, num_clauses,
                      complejidad, sat_heur, resultado):
    base = filename
    for ext in ['.xz', '.cnf']:
        if base.endswith(ext):
            base = base[:-len(ext)]
    output = f"{base}_DIMACS_resultado.txt"
    pct_heur = sat_heur * 100.0 / num_clauses if num_clauses > 0 else 100
    lineas = [
        "c " + "="*50,
        "c SAT Solver - Dinamica Polinomial de Quaterniones",
        "c O(log n) + CDCL (PySAT) = SAT/UNSAT CORRECTO",
        "c " + "="*50,
        f"c Archivo: {filename}",
        f"c Variables: {num_vars}",
        f"c Clausulas: {num_clauses}",
        f"c Complejidad O(log n): {complejidad:.6f}",
        f"c Clausulas satisfechas (heuristico): {sat_heur}/{num_clauses} ({pct_heur:.2f}%)",
        "c",
        f"s {resultado}"
    ]
    if resultado == "SAT" and asig_final is not None:
        lits = []
        for v in sorted(asig_final.keys()):
            lits.append(str(v) if asig_final[v] else str(-v))
        for i in range(0, len(lits), 15):
            chunk = lits[i:i+15]
            if i + 15 >= len(lits):
                lineas.append("v " + " ".join(chunk) + " 0")
            else:
                lineas.append("v " + " ".join(chunk))
    texto = "\n".join(lineas)
    with open(output, 'w') as f:
        f.write(texto)
    return output, texto

# ============================================================
# 3) Navegador Universal Optimizado - Modo Bitcoin
# ============================================================

class SHA256UniversalNavigatorFast:
    def __init__(self):
        self.var_map_by_num = {}
        self.msg_vars = {}   # (w,b) -> var
        self.hash_vars = {}  # (w,b) -> var
        self.assignments = {}
        self.clauses = []
        self.implications = defaultdict(set)
        self.original_msg = None
        self.original_hash = None

    def load_map(self, content):
        for line in content.splitlines():
            if ':' not in line:
                continue
            try:
                name, num_str = line.strip().split(':')
                num = int(num_str)
            except:
                continue
            self.var_map_by_num[num] = name.strip()
            m = re.match(r'msg_w(\d+)_b(\d+)', name)
            if m:
                self.msg_vars[(int(m.group(1)), int(m.group(2)))] = num
            m = re.match(r'hash_w(\d+)_b(\d+)', name)
            if m:
                self.hash_vars[(int(m.group(1)), int(m.group(2)))] = num
        print(f"✅ Map: {len(self.msg_vars)} msg bits, {len(self.hash_vars)} hash bits")

    def load_sat_result(self, content):
        for line in content.splitlines():
            line = line.strip()
            if not line.startswith('v'):
                continue
            parts = line.split()
            for lit_str in parts[1:-1]:  # EXACTO como tu parser
                try:
                    lit_val = int(lit_str)
                    var_num = abs(lit_val)
                    value = 1 if lit_val > 0 else 0
                    self.assignments[var_num] = value
                except:
                    continue
        print(f"✅ Asignaciones SAT: {len(self.assignments)} vars")
        self._compute_original_msg_hash()

    def _compute_original_msg_hash(self):
        msg_bits = {}
        for var, name in self.var_map_by_num.items():
            if var not in self.assignments:
                continue
            if name.startswith("msg_w"):
                parts = name.split('_')
                w_idx = int(parts[1][1:])
                b_idx = int(parts[2][1:])
                msg_bits[(w_idx, b_idx)] = self.assignments[var]
        words = [0] * 16
        for w in range(16):
            for b in range(32):
                if msg_bits.get((w, b), 0) == 1:
                    words[w] |= (1 << b)
        self.original_msg = b''.join(w.to_bytes(4, 'big') for w in words)
        self.original_hash = hashlib.sha256(self.original_msg).hexdigest()
        print("\n📝 Mensaje original:", self.original_msg.hex())
        print("🔐 Hash original:", self.original_hash)

    def load_cnf(self, content):
        clause = []
        for line in content.splitlines():
            line = line.strip()
            if not line or line[0] in 'cp%':
                continue
            for tok in line.split():
                try:
                    lit = int(tok)
                except:
                    continue
                if lit == 0:
                    if clause:
                        self.clauses.append(tuple(clause))
                        if len(clause) == 2:
                            a, b = clause
                            self.implications[-a].add(b)
                            self.implications[-b].add(a)
                        clause = []
                else:
                    clause.append(lit)
        print(f"✅ CNF: {len(self.clauses)} cláusulas, {sum(len(v) for v in self.implications.values())} implicaciones")

    def _propagate_implications(self, working):
        q = deque()
        for var, val in working.items():
            lit = var if val == 1 else -var
            q.append(lit)
        propagated = 0
        while q:
            lit = q.popleft()
            for impl in self.implications.get(lit, ()):
                v = abs(impl)
                val = 1 if impl > 0 else 0
                if v not in working:
                    working[v] = val
                    propagated += 1
                    q.append(impl)
        return propagated

    def _extract_message_from(self, assign):
        words = [0] * 16
        for (w, b), var in self.msg_vars.items():
            if assign.get(var, 0) == 1:
                words[w] |= (1 << b)
        return b''.join(w.to_bytes(4, 'big') for w in words)

    def _hash_match_bits(self, h1, h2):
        b1 = bytes.fromhex(h1)
        b2 = bytes.fromhex(h2)
        return sum(8 - bin(a ^ b).count('1') for a, b in zip(b1, b2))

    def _count_leading_zero_bits(self, h_hex):
        b = bytes.fromhex(h_hex)
        count = 0
        for byte in b:
            if byte == 0:
                count += 8
            else:
                for i in range(7, -1, -1):
                    if ((byte >> i) & 1) == 0:
                        count += 1
                    else:
                        return count
                break
        return count

    def _local_search_flips(self, base_assign, score_func,
                            max_flips_per_try=6, max_tries=300):
        import random
        best_assign = dict(base_assign)
        best_msg = self._extract_message_from(best_assign)
        best_hash = hashlib.sha256(best_msg).hexdigest()
        best_score = score_func(best_hash)
        msg_var_list = [var for (_, _), var in self.msg_vars.items()]
        print(f"   [Local search] score inicial: {best_score}")
        for t in range(max_tries):
            trial = dict(base_assign)
            n_flips = random.randint(1, max_flips_per_try)
            flip_vars = random.sample(msg_var_list, n_flips)
            for v in flip_vars:
                orig = self.assignments.get(v, 0)
                trial[v] = 1 - orig if random.random() < 0.7 else orig
            msg = self._extract_message_from(trial)
            h = hashlib.sha256(msg).hexdigest()
            s = score_func(h)
            if s > best_score:
                best_score = s
                best_msg = msg
                best_hash = h
                best_assign = trial
                print(f"   [Local] nuevo mejor score: {s} (try {t+1})")
            if t % 200 == 0 and t > 0:
                pass
        return best_msg, best_hash, best_score

    def find_bitcoin_preimage(self, difficulty_bits,
                              max_outer_loops=200, inner_tries=2000):
        difficulty_bits = int(difficulty_bits)
        print("\n" + "="*60)
        print(f"⛏️  BUSCANDO PREIMAGEN BITCOIN-LIKE")
        print(f"   Dificultad: {difficulty_bits} bits de ceros iniciales")
        print("="*60)
        if not (1 <= difficulty_bits <= 256):
            print("❌ dificultad debe estar entre 1 y 256")
            return None

        orig_zeros = self._count_leading_zero_bits(self.original_hash)
        print(f"   Hash original: {self.original_hash} ({orig_zeros} ceros iniciales)")
        if orig_zeros >= difficulty_bits:
            print("✅ El mensaje original ya cumple la dificultad")
            print("   Mensaje:", self.original_msg.hex())
            return self.original_msg

        # target_hex solo para compatibilidad; dificultad se mide con count_leading_zero_bits
        target_hex = "0" * 64
        target_bytes = bytes.fromhex(target_hex)
        target_words = [int.from_bytes(target_bytes[i*4:(i+1)*4], 'big') for i in range(8)]

        for loop in range(1, max_outer_loops + 1):
            print(f"\n🌀 Iteración externa {loop}/{max_outer_loops}")

            # 1) Fijar SOLO los primeros 'difficulty_bits' bits MSB del hash a 0
            print("[1/3] Fijando bits de hash objetivo (leading zeros)...")
            working = {}
            bits_fijados = 0
            for global_bit in range(difficulty_bits):
                word_index = global_bit // 32          # 0..7
                bit_in_word_msb = global_bit % 32      # 0..31 desde MSB
                bit_in_word_lsb = 31 - bit_in_word_msb # map a LSB-first
                var = self.hash_vars.get((word_index, bit_in_word_lsb))
                if var:
                    working[var] = 0
                    bits_fijados += 1
            print(f"   Bits fijados a 0: {bits_fijados}/{difficulty_bits}")

            # 2) Propagación rápida
            print("[2/3] Propagación rápida (implicaciones)...")
            propagated = self._propagate_implications(working)
            print(f"   {propagated} nuevas variables fijadas")

            msg_det = sum(1 for v in self.msg_vars.values() if v in working)
            for (w, b), var in self.msg_vars.items():
                if var not in working:
                    working[var] = self.assignments.get(var, 0)
            print(f"   Bits de mensaje determinados antes de completar: {msg_det}/512")

            base_msg = self._extract_message_from(working)
            base_hash = hashlib.sha256(base_msg).hexdigest()
            base_zeros = self._count_leading_zero_bits(base_hash)
            print("\n   Hash base:", base_hash)
            print(f"   Ceros iniciales base: {base_zeros}")

            if base_zeros >= difficulty_bits:
                print("\n🎉 ¡Mensaje que cumple dificultad encontrado sin búsqueda local!")
                print("   Mensaje:", base_msg.hex())
                return base_msg

            # 3) Búsqueda local optimizando ceros iniciales
            print("[3/3] Búsqueda local (mejorando leading zeros)...")
            def score_func(h_hex):
                return self._count_leading_zero_bits(h_hex)

            best_msg, best_hash, best_score = self._local_search_flips(
                working, score_func,
                max_flips_per_try=6,
                max_tries=inner_tries
            )

            print("\n   Mejor en esta iteración:")
            print("     Hash:", best_hash)
            print(f"     Ceros iniciales: {best_score}")

            if best_score >= difficulty_bits:
                print("\n🎉 ¡Mensaje que cumple dificultad encontrado!")
                print("   Mensaje:", best_msg.hex())
                return best_msg

        print("\n⚠️ No se logró alcanzar la dificultad tras varias iteraciones.")
        return None

# ============================================================
# 4) PIPELINE: construir, resolver y modo Bitcoin
# ============================================================

print("="*60)
print("  SHA-256 SAT PIPELINE")
print("  CNFBuilder + SATSolver + Universal Navigator (Bitcoin-like)")
print("="*60)

# 1) Construir CNF
builder = CNFBuilder()
digest = builder.build_sha256()
builder.save_dimacs("sha256_full.cnf")

# 2) Leer CNF
with open("sha256_full.cnf", 'r') as f:
    cnf_text = f.read()

# 3) Resolver con SATSolverQuaternion
print("\nResolviendo sha256_full.cnf con SATSolverQuaternion + PySAT...")
solver = SATSolverQuaternion()
ok = solver.parse_cnf_content(cnf_text, "sha256_full.cnf")
if not ok:
    print("❌ Error al parsear CNF")
else:
    print(f"  Variables: {solver.num_vars}")
    print(f"  Clausulas: {len(solver.clauses)}")
    t0 = time.time()
    resultado, asig_heur, complejidad, asig_exact = solver.solve()
    t1 = time.time()
    sat_heur = solver.contar_satisfechas(asig_heur)
    if resultado == "SAT":
        asig_final = asig_exact if asig_exact is not None else asig_heur
    else:
        asig_final = None
    print("\n" + "="*50)
    print(f"  RESULTADO SAT: {resultado}")
    print(f"  Tiempo solver: {t1 - t0:.2f}s")
    print(f"  Complejidad O(log n): {complejidad:.6f}")
    print(f"  Clausulas satisfechas (heuristico): {sat_heur}/{len(solver.clauses)}")
    print("="*50)

    if resultado == "SAT" and asig_final is not None:
        out_name, dimacs_text = guardar_resultado(
            "sha256_full.cnf", asig_final, solver.num_vars,
            len(solver.clauses), complejidad, sat_heur, resultado
        )
        print(f"\nArchivo resultado DIMACS: {out_name}")

        # 4) Cargar todo en el navegador universal
        with open("sha256_full.cnf.map", 'r') as f:
            map_text = f.read()

        nav = SHA256UniversalNavigatorFast()
        nav.load_map(map_text)
        nav.load_sat_result(dimacs_text)
        nav.load_cnf(cnf_text)

        print("\n" + "="*60)
        print("⛏️  BUSCAR PREIMÁGENES TIPO BITCOIN (leading zeros)")
        print("="*60)
        while True:
            diff = input("\nBits de dificultad (leading zeros, ej. 20, 32, 40, 80) o 'q' para salir: ").strip()
            if diff.lower() == 'q':
                break
            try:
                difficulty_bits = int(diff)
            except:
                print("❌ No es un entero válido")
                continue
            if not (1 <= difficulty_bits <= 256):
                print("❌ Debe estar entre 1 y 256")
                continue

            msg = nav.find_bitcoin_preimage(
                difficulty_bits,
                max_outer_loops=200,   # puedes bajar/subir según lo agresivo que quieras
                inner_tries=2000       # idem
            )
            if msg:
                final_hash = hashlib.sha256(msg).hexdigest()
                print("\n✅ MENSAJE ENCONTRADO:")
                print("   Mensaje:", msg.hex())
                print("   Hash   :", final_hash)
                print("   Leading zeros:", nav._count_leading_zero_bits(final_hash))
    else:
        print("❌ No se obtuvo un modelo SAT, no se puede navegar.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.5 MB/s eta 0:00:00
  SHA-256 SAT PIPELINE
  CNFBuilder + SATSolver + Universal Navigator (Bitcoin-like)
Generando variables de entrada (512 bits)...
Expandiendo Message Schedule (W0..W63)...
  ...Ronda de expansión 20/64
  ...Ronda de expansión 25/64
  ...Ronda de expansión 30/64
  ...Ronda de expansión 35/64
  ...Ronda de expansión 40/64
  ...Ronda de expansión 45/64
  ...Ronda de expansión 50/64
  ...Ronda de expansión 55/64
  ...Ronda de expansión 60/64
Inicializando estado hash...
Ejecutando Compresión (64 Rondas)...
  ...Compresión ronda 0/64
  ...Compresión ronda 10/64
  ...Compresión ronda 20/64
  ...Compresión ronda 30/64
  ...Compresión ronda 40/64
  ...Compresión ronda 50/64
  ...Compresión ronda 60/64
Suma final con estado inicial...
¡Hecho! Generadas 116328 variables y 401408 cláusulas.
Guardando en sha256_full.cnf...

Resolviendo sha256_full.cnf con SATSolverQuaternion + PySAT...
  Variables: 116328
  Clausulas: 401

KeyboardInterrupt: Interrupted by user

In [ ]:
#@title ⛏️ BITCOIN 100%: SHA256d(80B) CNF (universal) + SAT model + PoW mining by target
!pip install python-sat -q

import numpy as np
import math
import hashlib
import re
import time
from collections import defaultdict
from pysat.solvers import Solver as PySolver

# ============================================================
# Utils Bitcoin
# ============================================================

def sha256d(b: bytes) -> bytes:
    return hashlib.sha256(hashlib.sha256(b).digest()).digest()

def bits_compact_to_target_int(bits32: int) -> int:
    """
    Bitcoin compact bits -> target integer.
    bits32: 0x1d00ffff style.
    """
    exp = (bits32 >> 24) & 0xff
    mant = bits32 & 0x007fffff  # mantisa 23 bits (sign bit ignored)
    if exp <= 3:
        target = mant >> (8 * (3 - exp))
    else:
        target = mant << (8 * (exp - 3))
    return target

def target_int_to_hex_be(target_int: int) -> str:
    return target_int.to_bytes(32, 'big').hex()

def hash_int_le(hash_bytes: bytes) -> int:
    return int.from_bytes(hash_bytes, 'little')

def hash_display_hex(hash_bytes: bytes) -> str:
    # Así lo muestran block explorers: reverse bytes
    return hash_bytes[::-1].hex()

# ============================================================
# CNF Builder for SHA-256 compression + multi-block SHA256 + SHA256d
#   Bit representation per word: list of 32 vars, LSB-first (bit0 = 2^0)
#   Bytes reconstruction for a word: word_int.to_bytes(4,'big')
# ============================================================

class CNFBuilderSHA256d80:
    def __init__(self):
        self.var_count = 0
        self.clauses = []
        self.var_map = {}

        self.K = [
            0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
            0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
            0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
            0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
            0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
            0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
            0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
            0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
        ]
        self.H_init = [
            0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
            0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
        ]

    def new_var(self, name=None):
        self.var_count += 1
        if name:
            self.var_map[name] = self.var_count
        return self.var_count

    def add_clause(self, lits):
        self.clauses.append(lits)

    # --- gates ---
    def gate_xor(self, a, b):
        c = self.new_var()
        self.add_clause([-a, -b, -c])
        self.add_clause([a, b, -c])
        self.add_clause([a, -b, c])
        self.add_clause([-a, b, c])
        return c

    def gate_and(self, a, b):
        c = self.new_var()
        self.add_clause([-a, -b, c])
        self.add_clause([a, -c])
        self.add_clause([b, -c])
        return c

    def gate_not(self, a):
        c = self.new_var()
        self.add_clause([-a, -c])
        self.add_clause([a, c])
        return c

    def gate_maj(self, a, b, c):
        out = self.new_var()
        self.add_clause([-a, -b, out])
        self.add_clause([-a, -c, out])
        self.add_clause([-b, -c, out])
        self.add_clause([a, b, -out])
        self.add_clause([a, c, -out])
        self.add_clause([b, c, -out])
        return out

    def gate_ch(self, e, f, g):
        out = self.new_var()
        self.add_clause([-e, -f, out])
        self.add_clause([-e, f, -out])
        self.add_clause([e, -g, out])
        self.add_clause([e, g, -out])
        return out

    # --- 32-bit words, bit lists LSB-first ---
    def const_word(self, val):
        bits = []
        for i in range(32):
            bit_val = (val >> i) & 1
            v = self.new_var()
            self.add_clause([v] if bit_val else [-v])
            bits.append(v)
        return bits

    def xor_word(self, w1, w2):
        return [self.gate_xor(a, b) for a, b in zip(w1, w2)]

    def and_word(self, w1, w2):
        return [self.gate_and(a, b) for a, b in zip(w1, w2)]

    def rot_right(self, w, n):
        return w[n:] + w[:n]

    def shift_right(self, w, n):
        zeros = []
        for _ in range(n):
            z = self.new_var()
            self.add_clause([-z])  # force 0
            zeros.append(z)
        return w[n:] + zeros

    def add_word(self, w1, w2):
        result = []
        carry = None
        for i in range(32):
            a = w1[i]
            b = w2[i]
            if carry is None:
                s = self.gate_xor(a, b)
                c_out = self.gate_and(a, b)
            else:
                tmp = self.gate_xor(a, b)
                s = self.gate_xor(tmp, carry)
                ab = self.gate_and(a, b)
                cin_xor = self.gate_and(carry, tmp)
                c_out = self.new_var()
                self.add_clause([-ab, c_out])
                self.add_clause([-cin_xor, c_out])
                self.add_clause([ab, cin_xor, -c_out])
            result.append(s)
            carry = c_out
        return result

    def sigma0(self, w):
        r7 = self.rot_right(w, 7)
        r18 = self.rot_right(w, 18)
        s3 = self.shift_right(w, 3)
        return self.xor_word(self.xor_word(r7, r18), s3)

    def sigma1(self, w):
        r17 = self.rot_right(w, 17)
        r19 = self.rot_right(w, 19)
        s10 = self.shift_right(w, 10)
        return self.xor_word(self.xor_word(r17, r19), s10)

    def Sigma0(self, w):
        r2 = self.rot_right(w, 2)
        r13 = self.rot_right(w, 13)
        r22 = self.rot_right(w, 22)
        return self.xor_word(self.xor_word(r2, r13), r22)

    def Sigma1(self, w):
        r6 = self.rot_right(w, 6)
        r11 = self.rot_right(w, 11)
        r25 = self.rot_right(w, 25)
        return self.xor_word(self.xor_word(r6, r11), r25)

    def Maj_word(self, x, y, z):
        return [self.gate_maj(a, b, c) for a, b, c in zip(x, y, z)]

    def Ch_word(self, x, y, z):
        return [self.gate_ch(a, b, c) for a, b, c in zip(x, y, z)]

    def expand_W(self, M16):
        W = [None] * 64
        for i in range(16):
            W[i] = M16[i]
        for i in range(16, 64):
            s1 = self.sigma1(W[i-2])
            s0 = self.sigma0(W[i-15])
            t1 = self.add_word(s1, W[i-7])
            t2 = self.add_word(s0, W[i-16])
            W[i] = self.add_word(t1, t2)
        return W

    def compress_block(self, M16, H_state):
        """
        M16: list of 16 words (LSB-first bits)
        H_state: list of 8 words (LSB-first bits) current H
        returns new_H_state (8 words)
        """
        W = self.expand_W(M16)

        a, b, c, d, e, f, g, h = H_state

        for i in range(64):
            S1_e = self.Sigma1(e)
            ch = self.Ch_word(e, f, g)
            ki = self.const_word(self.K[i])

            sum1 = self.add_word(h, S1_e)
            sum2 = self.add_word(ch, ki)
            sum3 = self.add_word(sum1, sum2)
            T1 = self.add_word(sum3, W[i])

            S0_a = self.Sigma0(a)
            maj = self.Maj_word(a, b, c)
            T2 = self.add_word(S0_a, maj)

            h = g
            g = f
            f = e
            e = self.add_word(d, T1)
            d = c
            c = b
            b = a
            a = self.add_word(T1, T2)

        final_work = [a, b, c, d, e, f, g, h]
        new_H = []
        for i in range(8):
            new_H.append(self.add_word(H_state[i], final_work[i]))
        return new_H

    def build_sha256d_header80(self):
        """
        Input: 80 bytes header => 20 words variables hdr_w0..hdr_w19 (LSB-first bits)
        SHA256(header) uses 2 blocks (512 + 512 with padding)
        SHA256(second) uses 1 block (digest 32 bytes + padding)
        Output: hash_w0..hash_w7 bits (final digest), universal CNF (no objective).
        """
        # 80 bytes => 20 words
        hdr_words = []
        for wi in range(20):
            wbits = []
            for bi in range(32):
                v = self.new_var(f"hdr_w{wi}_b{bi}")
                wbits.append(v)
            hdr_words.append(wbits)

        # Block1: words 0..15
        block1 = hdr_words[0:16]

        # Block2: words 16..19 + padding words
        block2 = []
        block2.extend(hdr_words[16:20])  # 4 words (16 bytes)
        block2.append(self.const_word(0x80000000))  # word4 = 0x80 00 00 00
        for _ in range(5, 15):
            block2.append(self.const_word(0))
        block2.append(self.const_word(0))          # word14 high length
        block2.append(self.const_word(0x00000280)) # word15 low length = 640 bits = 0x280

        # First SHA256: init state
        H0 = [self.const_word(x) for x in self.H_init]
        H1 = self.compress_block(block1, H0)
        H2 = self.compress_block(block2, H1)  # digest1 words = H2

        # Second SHA256: message = digest1 (32 bytes) + padding, 1 block
        block3 = []
        block3.extend(H2)  # 8 words
        block3.append(self.const_word(0x80000000))  # word8
        for _ in range(9, 15):
            block3.append(self.const_word(0))
        block3.append(self.const_word(0))          # word14
        block3.append(self.const_word(0x00000100)) # word15 length = 256 bits

        H0b = [self.const_word(x) for x in self.H_init]
        H3 = self.compress_block(block3, H0b)  # final digest words

        # map output vars
        for wi in range(8):
            for bi, var in enumerate(H3[wi]):
                self.var_map[f"hash_w{wi}_b{bi}"] = var

        # also map nonce bits for convenience: nonce is last word (hdr_w19)
        for bi, var in enumerate(hdr_words[19]):
            self.var_map[f"nonce_b{bi}"] = var

        return hdr_words, H3

    def save_dimacs(self, filename):
        with open(filename, 'w') as f:
            f.write(f"p cnf {self.var_count} {len(self.clauses)}\n")
            for clause in self.clauses:
                f.write(" ".join(map(str, clause)) + " 0\n")
        with open(filename + ".map", "w") as f:
            for k, v in self.var_map.items():
                f.write(f"{k}:{v}\n")

# ============================================================
# 2) SAT solver híbrido (tu estilo) + PySAT exacto
# ============================================================

class SATSolverQuaternion:
    def __init__(self):
        self.num_vars = 0
        self.clauses = []
        self.var_pos = None
        self.var_neg = None
        self.var_clauses = None
        self.implications = None

    def parse_cnf_content(self, content):
        self.clauses = []
        max_var = 0
        for line in content.splitlines():
            line = line.strip()
            if not line or line[0] in 'cp%':
                if line.startswith('p cnf'):
                    parts = line.split()
                    if len(parts) >= 4:
                        self.num_vars = int(parts[2])
                continue
            clause = []
            for tok in line.split():
                try:
                    lit = int(tok)
                except:
                    continue
                if lit == 0:
                    if clause:
                        self.clauses.append(tuple(clause))
                        clause = []
                else:
                    clause.append(lit)
                    max_var = max(max_var, abs(lit))
            if clause:
                self.clauses.append(tuple(clause))
        self.num_vars = max(self.num_vars, max_var)
        self._build_structures()
        return len(self.clauses) > 0

    def _build_structures(self):
        n = self.num_vars
        self.var_pos = np.zeros(n + 1, dtype=np.float64)
        self.var_neg = np.zeros(n + 1, dtype=np.float64)
        self.var_clauses = [[] for _ in range(n + 1)]
        self.implications = {i: set() for i in range(-n, n + 1) if i != 0}

        for idx, clause in enumerate(self.clauses):
            peso = 1.0 / max(len(clause), 1)
            for lit in clause:
                v = abs(lit)
                if lit > 0:
                    self.var_pos[v] += peso
                else:
                    self.var_neg[v] += peso
                self.var_clauses[v].append(idx)
            if len(clause) == 2:
                a, b = clause
                self.implications[-a].add(b)
                self.implications[-b].add(a)

    def contar_satisfechas(self, asignaciones):
        count = 0
        for clause in self.clauses:
            for lit in clause:
                v = abs(lit)
                val = asignaciones.get(v, True)
                if (lit > 0 and val) or (lit < 0 and not val):
                    count += 1
                    break
        return count

    def _heuristic(self):
        n_vars = self.num_vars
        asig = {}

        # unit clauses
        for clause in self.clauses:
            if len(clause) == 1:
                lit = clause[0]
                asig[abs(lit)] = (lit > 0)

        # propagate binary implications a bit
        changed = True
        rounds = 0
        max_rounds = int(np.log2(n_vars + 1)) + 2
        while changed and rounds < max_rounds:
            rounds += 1
            changed = False
            for var, val in list(asig.items()):
                lit = var if val else -var
                for impl in self.implications.get(lit, ()):
                    iv = abs(impl)
                    ival = (impl > 0)
                    if iv not in asig:
                        asig[iv] = ival
                        changed = True

        # fill rest with sign of (pos-neg)
        for v in range(1, n_vars + 1):
            if v in asig:
                continue
            asig[v] = (self.var_pos[v] >= self.var_neg[v])
        return asig

    def _pysat(self):
        s = PySolver(name='g3')
        for c in self.clauses:
            s.add_clause(list(c))
        ok = s.solve()
        if not ok:
            s.delete()
            return "UNSAT", None
        model = s.get_model()
        s.delete()
        asig = {abs(lit): (lit > 0) for lit in model}
        return "SAT", asig

    def solve(self):
        if not self.clauses:
            return "SAT", {}, 0.0, {}
        print("  [1] Heurístico...")
        asig_h = self._heuristic()
        sat_h = self.contar_satisfechas(asig_h)
        print(f"      Heurístico: {sat_h}/{len(self.clauses)}")
        print("  [2] PySAT exacto (CDCL)...")
        res, asig_e = self._pysat()
        return res, asig_h, np.log(self.num_vars + len(self.clauses) + 1), asig_e

def guardar_resultado(base, asig_final, num_vars, num_clauses, complejidad, sat_heur, resultado):
    output = f"{base}_DIMACS_resultado.txt"
    pct_heur = sat_heur * 100.0 / num_clauses if num_clauses else 100.0
    lines = [
        "c " + "="*50,
        "c SAT Solver - Result",
        "c " + "="*50,
        f"c Variables: {num_vars}",
        f"c Clausulas: {num_clauses}",
        f"c Complejidad O(log n): {complejidad:.6f}",
        f"c Clausulas satisfechas (heuristico): {sat_heur}/{num_clauses} ({pct_heur:.2f}%)",
        "c",
        f"s {resultado}"
    ]
    if resultado == "SAT" and asig_final:
        lits = []
        for v in sorted(asig_final.keys()):
            lits.append(str(v) if asig_final[v] else str(-v))
        for i in range(0, len(lits), 15):
            chunk = lits[i:i+15]
            lines.append("v " + " ".join(chunk) + (" 0" if i + 15 >= len(lits) else ""))
    text = "\n".join(lines)
    with open(output, "w") as f:
        f.write(text)
    return output, text

# ============================================================
# 3) Parser del modelo SAT -> header bytes + hash bytes + minería PoW real
# ============================================================

class BitcoinSATModel:
    def __init__(self, map_text: str, dimacs_result_text: str):
        self.name_by_var = {}
        self.var_by_name = {}
        for line in map_text.splitlines():
            if ':' not in line:
                continue
            k, v = line.strip().split(':', 1)
            try:
                v = int(v)
            except:
                continue
            self.var_by_name[k] = v
            self.name_by_var[v] = k

        self.assign = {}
        for line in dimacs_result_text.splitlines():
            line = line.strip()
            if not line.startswith("v "):
                continue
            parts = line.split()
            for lit_str in parts[1:-1]:  # igual a tu parser
                try:
                    lit = int(lit_str)
                except:
                    continue
                self.assign[abs(lit)] = (lit > 0)

    def _word_from_bits(self, prefix: str, wi: int) -> int:
        val = 0
        for bi in range(32):
            name = f"{prefix}{wi}_b{bi}"
            var = self.var_by_name.get(name)
            if var and self.assign.get(var, False):
                val |= (1 << bi)
        return val

    def get_header80_bytes(self) -> bytes:
        # hdr_w0..hdr_w19 -> 80 bytes
        b = bytearray()
        for wi in range(20):
            w = self._word_from_bits("hdr_w", wi)
            b.extend(w.to_bytes(4, 'big'))
        return bytes(b)

    def get_hash_bytes(self) -> bytes:
        b = bytearray()
        for wi in range(8):
            w = self._word_from_bits("hash_w", wi)
            b.extend(w.to_bytes(4, 'big'))
        return bytes(b)

# ============================================================
# 4) Pipeline: build CNF (sha256d 80B), solve once, then mine PoW
# ============================================================

print("="*70)
print("  ⛏️ BITCOIN 100% PIPELINE")
print("  CNF universal de SHA256d(header 80B) + SAT model + PoW target mining")
print("="*70)

t_build0 = time.time()
builder = CNFBuilderSHA256d80()
builder.build_sha256d_header80()
builder.save_dimacs("bitcoin_sha256d_80B.cnf")
t_build1 = time.time()

print(f"\nCNF generado: vars={builder.var_count}, clauses={len(builder.clauses)}")
print(f"Tiempo build CNF: {t_build1 - t_build0:.2f}s")

with open("bitcoin_sha256d_80B.cnf", "r") as f:
    cnf_text = f.read()
with open("bitcoin_sha256d_80B.cnf.map", "r") as f:
    map_text = f.read()

print("\nResolviendo CNF universal (sin target) UNA VEZ...")
solver = SATSolverQuaternion()
ok = solver.parse_cnf_content(cnf_text)
if not ok:
    raise RuntimeError("No se pudieron parsear cláusulas.")

print(f"CNF parseado: vars={solver.num_vars}, clauses={len(solver.clauses)}")
t0 = time.time()
res, asig_h, compl, asig_e = solver.solve()
t1 = time.time()
sat_heur = solver.contar_satisfechas(asig_h)

print("\n" + "="*50)
print(f"RESULTADO: {res}")
print(f"Tiempo solver: {t1 - t0:.2f}s")
print("="*50)

if res != "SAT" or asig_e is None:
    raise RuntimeError("UNSAT o sin modelo.")

out_file, dimacs_text = guardar_resultado(
    "bitcoin_sha256d_80B",
    asig_e,
    solver.num_vars,
    len(solver.clauses),
    compl,
    sat_heur,
    res
)
print(f"\nDIMACS resultado: {out_file}")

# ---- Parse model and verify SHA256d correctness ----
model = BitcoinSATModel(map_text, dimacs_text)
header80 = model.get_header80_bytes()
hash_from_cnf = model.get_hash_bytes()
hash_real = sha256d(header80)

print("\n" + "="*70)
print("✅ Verificación SHA256d(header80) con el modelo SAT")
print("="*70)
print("Header80 (hex):", header80.hex())
print("Hash CNF   (hex):", hash_from_cnf.hex())
print("Hash real  (hex):", hash_real.hex())
print("Hash display (rev):", hash_display_hex(hash_real))
print("Coinciden:", hash_from_cnf == hash_real)

# ============================================================
# Mining loop: REAL Bitcoin PoW (hash_le <= target(bits))
#   Cambia SOLO nonce (últimos 4 bytes)
# ============================================================

def mine_pow_by_bits(header80_base: bytes, bits_hex: str, max_seconds=10.0, report_every=50000):
    bits_hex = bits_hex.lower().strip()
    if not re.fullmatch(r"[0-9a-f]{8}", bits_hex):
        raise ValueError("bits_hex debe ser 8 hex (ej: 1d00ffff)")
    bits32 = int(bits_hex, 16)
    target = bits_compact_to_target_int(bits32)
    target_hex = target_int_to_hex_be(target)
    print("\n" + "="*70)
    print("⛏️  MINING (Bitcoin real): SHA256d(header80) <= target(bits)")
    print("="*70)
    print("bits:", bits_hex, "target_be:", target_hex)

    header = bytearray(header80_base)
    start_nonce = int.from_bytes(header[76:80], 'little', signed=False)  # nonce field in Bitcoin header is little-endian
    # OJO: nosotros tratamos el header como bytes "tal cual"; nonce little-endian es convención Bitcoin.
    # Igual puedes minar cambiando esos 4 bytes como little-endian.
    print("Nonce inicial (interpretado LE):", start_nonce)

    t0 = time.time()
    best = None
    best_hash = None
    best_leading_zeros = -1
    tries = 0

    nonce = start_nonce
    while True:
        # set nonce as little-endian in last 4 bytes (Bitcoin real)
        header[76:80] = nonce.to_bytes(4, 'little', signed=False)

        h = sha256d(bytes(header))
        h_int = int.from_bytes(h, 'little')
        tries += 1

        # leading zeros (solo para ver progreso)
        # (contar ceros MSB del hash display)
        zeros = 0
        for byte in h:
            if byte == 0:
                zeros += 8
            else:
                for i in range(7, -1, -1):
                    if ((byte >> i) & 1) == 0:
                        zeros += 1
                    else:
                        break
                break

        if zeros > best_leading_zeros:
            best_leading_zeros = zeros
            best = bytes(header)
            best_hash = h

        if h_int <= target:
            t1 = time.time()
            print("\n🎉 FOUND!")
            print("Tiempo:", t1 - t0, "s | intentos:", tries)
            print("Header80:", bytes(header).hex())
            print("Hash raw:", h.hex())
            print("Hash display:", hash_display_hex(h))
            print("Leading zeros (aprox):", zeros)
            return bytes(header), h

        if tries % report_every == 0:
            elapsed = time.time() - t0
            print(f"tries={tries} elapsed={elapsed:.1f}s bestZeros={best_leading_zeros} bestHashDisplay={hash_display_hex(best_hash)[:16]}...")
            if elapsed >= max_seconds:
                print("\n⏱️ Tiempo límite alcanzado.")
                print("Mejor hasta ahora:")
                print("bestZeros:", best_leading_zeros)
                print("bestHeader80:", best.hex())
                print("bestHash raw:", best_hash.hex())
                print("bestHash display:", hash_display_hex(best_hash))
                return None, None

        nonce = (nonce + 1) & 0xffffffff  # iterar

# ============================================================
# Interactive: ask bits and mine
# ============================================================

print("\n" + "="*70)
print("Entrada de minería:")
print("- bits compact (8 hex) ej GENESIS: 1d00ffff")
print("- OJO: minar genesis exacto en CPU puede ser mucho trabajo; usa max_seconds para probar.")
print("="*70)

while True:
    bits_hex = input("\nBITS compact (8 hex) o 'q': ").strip()
    if bits_hex.lower() == 'q':
        break

    max_s = input("max_seconds (ej 5, 10, 60): ").strip()
    try:
        max_seconds = float(max_s)
    except:
        max_seconds = 10.0

    try:
        found_header, found_hash = mine_pow_by_bits(header80, bits_hex, max_seconds=max_seconds, report_every=50000)
    except Exception as e:
        print("Error:", e)
        continue

    if found_header is not None:
        print("\n✅ SOLUCIÓN PoW encontrada")
        print("Header80:", found_header.hex())
        print("Hash display:", hash_display_hex(found_hash))
    else:
        print("\n⚠️ No encontrado en este tiempo. Sube max_seconds o baja dificultad.")

  ⛏️ BITCOIN 100% PIPELINE
  CNF universal de SHA256d(header 80B) + SAT model + PoW target mining

CNF generado: vars=347768, clauses=1203904
Tiempo build CNF: 2.83s

Resolviendo CNF universal (sin target) UNA VEZ...
CNF parseado: vars=347768, clauses=1203904
  [1] Heurístico...
      Heurístico: 895188/1203904
  [2] PySAT exacto (CDCL)...

RESULTADO: SAT
Tiempo solver: 59.93s

DIMACS resultado: bitcoin_sha256d_80B_DIMACS_resultado.txt

✅ Verificación SHA256d(header80) con el modelo SAT
Header80 (hex): ca1293b647042fc54957caa5a9519c282550a8058f0177c8594cd6c8a41b360401e319e25a4df5c9ee5ac84cb490b9004eee259043c270640fb24a219c919fd0e34ead29701ea25494426a174649173f
Hash CNF   (hex): ad14243c33920c7d648289614189b0c49c25ae30914d81c87e67ae55200b05a7
Hash real  (hex): 20e699c146b0590cdaf52eea5b9e8b446bc850344bea3bc2d6849a4dba3604b3
Hash display (rev): b30436ba4d9a84d6c23bea4b3450c86b448b9e5bea2ef5da0c59b046c199e620
Coinciden: False

Entrada de minería:
- bits compact (8 hex) ej GENESIS: 1d00fff

KeyboardInterrupt: 

In [ ]:
#@title ⛏️ Bitcoin 100%: SHA256d(80B) CNF universal + SAT (phase-hints) + PoW target mining
!pip install python-sat -q

import hashlib
import re
import time
from collections import defaultdict, deque
from pysat.solvers import Solver as PySolver

# ============================================================
# Utils Bitcoin
# ============================================================

def sha256d(b: bytes) -> bytes:
    return hashlib.sha256(hashlib.sha256(b).digest()).digest()

def bits_compact_to_target_int(bits32: int) -> int:
    exp = (bits32 >> 24) & 0xff
    mant = bits32 & 0x007fffff
    if exp <= 3:
        return mant >> (8 * (3 - exp))
    return mant << (8 * (exp - 3))

def target_int_to_hex_be(target_int: int) -> str:
    return target_int.to_bytes(32, 'big').hex()

def hash_display_hex(hash_bytes: bytes) -> str:
    return hash_bytes[::-1].hex()

def count_leading_zero_bits_from_raw_sha256_bytes(h: bytes) -> int:
    # cuenta ceros MSB-first en bytes del digest (big-endian view)
    count = 0
    for byte in h:
        if byte == 0:
            count += 8
        else:
            for i in range(7, -1, -1):
                if ((byte >> i) & 1) == 0:
                    count += 1
                else:
                    return count
            return count
    return count

# ============================================================
# CNF Builder: SHA256d(header80)
#   - words are LSB-first bitlists
#   - header80 = 20 words (80 bytes)
# ============================================================

class CNFBuilderSHA256d80:
    def __init__(self):
        self.var_count = 0
        self.clauses = []
        self.var_map = {}

        self.K = [
            0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
            0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
            0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
            0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
            0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
            0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
            0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
            0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
        ]
        self.H_init = [
            0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
            0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
        ]

    def new_var(self, name=None):
        self.var_count += 1
        if name:
            self.var_map[name] = self.var_count
        return self.var_count

    def add_clause(self, lits):
        self.clauses.append(lits)

    def gate_xor(self, a, b):
        c = self.new_var()
        self.add_clause([-a, -b, -c])
        self.add_clause([a, b, -c])
        self.add_clause([a, -b, c])
        self.add_clause([-a, b, c])
        return c

    def gate_and(self, a, b):
        c = self.new_var()
        self.add_clause([-a, -b, c])
        self.add_clause([a, -c])
        self.add_clause([b, -c])
        return c

    def gate_maj(self, a, b, c):
        out = self.new_var()
        self.add_clause([-a, -b, out])
        self.add_clause([-a, -c, out])
        self.add_clause([-b, -c, out])
        self.add_clause([a, b, -out])
        self.add_clause([a, c, -out])
        self.add_clause([b, c, -out])
        return out

    def gate_ch(self, e, f, g):
        out = self.new_var()
        self.add_clause([-e, -f, out])
        self.add_clause([-e, f, -out])
        self.add_clause([e, -g, out])
        self.add_clause([e, g, -out])
        return out

    def const_word(self, val):
        bits = []
        for i in range(32):
            bit_val = (val >> i) & 1
            v = self.new_var()
            self.add_clause([v] if bit_val else [-v])
            bits.append(v)
        return bits

    def rot_right(self, w, n):
        return w[n:] + w[:n]

    def shift_right(self, w, n):
        zeros = []
        for _ in range(n):
            z = self.new_var()
            self.add_clause([-z])
            zeros.append(z)
        return w[n:] + zeros

    def xor_word(self, w1, w2):
        return [self.gate_xor(a, b) for a, b in zip(w1, w2)]

    def add_word(self, w1, w2):
        result = []
        carry = None
        for i in range(32):
            a = w1[i]
            b = w2[i]
            if carry is None:
                s = self.gate_xor(a, b)
                c_out = self.gate_and(a, b)
            else:
                tmp = self.gate_xor(a, b)
                s = self.gate_xor(tmp, carry)
                ab = self.gate_and(a, b)
                cin_xor = self.gate_and(carry, tmp)
                c_out = self.new_var()
                self.add_clause([-ab, c_out])
                self.add_clause([-cin_xor, c_out])
                self.add_clause([ab, cin_xor, -c_out])
            result.append(s)
            carry = c_out
        return result

    def sigma0(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 7), self.rot_right(w, 18)),
                             self.shift_right(w, 3))

    def sigma1(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 17), self.rot_right(w, 19)),
                             self.shift_right(w, 10))

    def Sigma0(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 2), self.rot_right(w, 13)),
                             self.rot_right(w, 22))

    def Sigma1(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 6), self.rot_right(w, 11)),
                             self.rot_right(w, 25))

    def Maj_word(self, x, y, z):
        return [self.gate_maj(a, b, c) for a, b, c in zip(x, y, z)]

    def Ch_word(self, x, y, z):
        return [self.gate_ch(a, b, c) for a, b, c in zip(x, y, z)]

    def expand_W(self, M16):
        W = [None] * 64
        for i in range(16):
            W[i] = M16[i]
        for i in range(16, 64):
            s1 = self.sigma1(W[i-2])
            s0 = self.sigma0(W[i-15])
            t1 = self.add_word(s1, W[i-7])
            t2 = self.add_word(s0, W[i-16])
            W[i] = self.add_word(t1, t2)
        return W

    def compress_block(self, M16, H_state):
        W = self.expand_W(M16)
        a, b, c, d, e, f, g, h = H_state

        for i in range(64):
            S1_e = self.Sigma1(e)
            ch = self.Ch_word(e, f, g)
            ki = self.const_word(self.K[i])

            sum1 = self.add_word(h, S1_e)
            sum2 = self.add_word(ch, ki)
            sum3 = self.add_word(sum1, sum2)
            T1 = self.add_word(sum3, W[i])

            S0_a = self.Sigma0(a)
            maj = self.Maj_word(a, b, c)
            T2 = self.add_word(S0_a, maj)

            h = g
            g = f
            f = e
            e = self.add_word(d, T1)
            d = c
            c = b
            b = a
            a = self.add_word(T1, T2)

        final_work = [a, b, c, d, e, f, g, h]
        new_H = []
        for i in range(8):
            new_H.append(self.add_word(H_state[i], final_work[i]))
        return new_H

    def build_sha256d_header80(self):
        # header80 = 20 words
        hdr_words = []
        for wi in range(20):
            wbits = []
            for bi in range(32):
                v = self.new_var(f"hdr_w{wi}_b{bi}")
                wbits.append(v)
            hdr_words.append(wbits)

        # block1: words 0..15
        block1 = hdr_words[0:16]

        # block2: words 16..19 + padding (80 bytes total)
        block2 = []
        block2.extend(hdr_words[16:20])
        block2.append(self.const_word(0x80000000))
        for _ in range(5, 15):
            block2.append(self.const_word(0))
        block2.append(self.const_word(0))
        block2.append(self.const_word(0x00000280))  # 80*8 = 640 bits

        H0 = [self.const_word(x) for x in self.H_init]
        H1 = self.compress_block(block1, H0)
        H2 = self.compress_block(block2, H1)  # digest1

        # second SHA256: message = digest1(32 bytes) + padding (1 block)
        block3 = []
        block3.extend(H2)  # 8 words
        block3.append(self.const_word(0x80000000))
        for _ in range(9, 15):
            block3.append(self.const_word(0))
        block3.append(self.const_word(0))
        block3.append(self.const_word(0x00000100))  # 256 bits

        H0b = [self.const_word(x) for x in self.H_init]
        H3 = self.compress_block(block3, H0b)  # final digest words

        for wi in range(8):
            for bi, var in enumerate(H3[wi]):
                self.var_map[f"hash_w{wi}_b{bi}"] = var
        for bi, var in enumerate(hdr_words[19]):
            self.var_map[f"nonce_b{bi}"] = var

        return hdr_words, H3

    def save_dimacs(self, filename):
        with open(filename, 'w') as f:
            f.write(f"p cnf {self.var_count} {len(self.clauses)}\n")
            for clause in self.clauses:
                f.write(" ".join(map(str, clause)) + " 0\n")
        with open(filename + ".map", "w") as f:
            for k, v in self.var_map.items():
                f.write(f"{k}:{v}\n")

# ============================================================
#  SAT solve with heuristic-100 adapted = PHASE HINTS
#   (fast, no reparación lenta)
# ============================================================

def parse_cnf_and_build_phases(cnf_text: str):
    """
    Parse DIMACS -> clauses + phase hints (SATSolver100-style weights).
    Returns: (nvars, clauses, phases)
    phases: list of literals, one per var, telling preferred polarity.
    """
    nvars = 0
    clauses = []
    clause = []

    # weights
    pos = None
    neg = None

    # forced by unit clauses
    forced = {}

    for line in cnf_text.splitlines():
        line = line.strip()
        if not line or line[0] in 'c%':
            continue
        if line.startswith('p cnf'):
            parts = line.split()
            nvars = int(parts[2])
            pos = [0.0] * (nvars + 1)
            neg = [0.0] * (nvars + 1)
            continue

        for tok in line.split():
            try:
                lit = int(tok)
            except:
                continue
            if lit == 0:
                if clause:
                    clauses.append(clause)
                    # weight update
                    w = 1.0 / (len(clause) ** 1.5)
                    for L in clause:
                        v = abs(L)
                        if L > 0:
                            pos[v] += w
                        else:
                            neg[v] += w
                    # unit forcing
                    if len(clause) == 1:
                        L = clause[0]
                        forced[abs(L)] = (L > 0)
                    clause = []
            else:
                clause.append(lit)

    if clause:
        clauses.append(clause)
        w = 1.0 / (len(clause) ** 1.5)
        for L in clause:
            v = abs(L)
            if L > 0:
                pos[v] += w
            else:
                neg[v] += w
        if len(clause) == 1:
            L = clause[0]
            forced[abs(L)] = (L > 0)

    # build phases list
    phases = []
    for v in range(1, nvars + 1):
        if v in forced:
            phases.append(v if forced[v] else -v)
        else:
            # pure literal hint
            if pos[v] > 0 and neg[v] == 0:
                phases.append(v)
            elif neg[v] > 0 and pos[v] == 0:
                phases.append(-v)
            else:
                phases.append(v if pos[v] >= neg[v] else -v)

    return nvars, clauses, phases

def solve_with_pysat_phases(nvars, clauses, phases, solver_name='g3'):
    s = PySolver(name=solver_name)
    for c in clauses:
        s.add_clause(c)

    # apply phase hints if supported
    try:
        s.set_phases(phases)
        used_phases = True
    except Exception:
        used_phases = False

    t0 = time.time()
    ok = s.solve()
    t1 = time.time()

    if not ok:
        s.delete()
        return "UNSAT", None, (t1 - t0), used_phases

    model = s.get_model()
    s.delete()
    asig = {abs(lit): (lit > 0) for lit in model}
    return "SAT", asig, (t1 - t0), used_phases

def save_dimacs_result(base, asig_final, nvars):
    out = f"{base}_DIMACS_resultado.txt"
    lines = ["s SAT"]
    lits = []
    for v in range(1, nvars + 1):
        val = asig_final.get(v, True)
        lits.append(str(v if val else -v))
    for i in range(0, len(lits), 15):
        chunk = lits[i:i+15]
        if i + 15 >= len(lits):
            lines.append("v " + " ".join(chunk) + " 0")
        else:
            lines.append("v " + " ".join(chunk))
    text = "\n".join(lines)
    with open(out, "w") as f:
        f.write(text)
    return out, text

# ============================================================
# Model parser for bitcoin CNF
# ============================================================

class BitcoinSATModel:
    def __init__(self, map_text: str, dimacs_result_text: str):
        self.var_by_name = {}
        for line in map_text.splitlines():
            if ':' not in line:
                continue
            k, v = line.strip().split(':', 1)
            try:
                v = int(v)
            except:
                continue
            self.var_by_name[k] = v

        self.assign = {}
        for line in dimacs_result_text.splitlines():
            line = line.strip()
            if not line.startswith("v "):
                continue
            parts = line.split()
            for lit_str in parts[1:-1]:
                try:
                    lit = int(lit_str)
                except:
                    continue
                self.assign[abs(lit)] = (lit > 0)

    def _word_from_bits(self, prefix: str, wi: int) -> int:
        val = 0
        for bi in range(32):
            var = self.var_by_name.get(f"{prefix}{wi}_b{bi}")
            if var and self.assign.get(var, False):
                val |= (1 << bi)
        return val

    def get_header80_bytes(self) -> bytes:
        b = bytearray()
        for wi in range(20):
            w = self._word_from_bits("hdr_w", wi)
            b.extend(w.to_bytes(4, 'big'))
        return bytes(b)

    def get_hash_bytes(self) -> bytes:
        b = bytearray()
        for wi in range(8):
            w = self._word_from_bits("hash_w", wi)
            b.extend(w.to_bytes(4, 'big'))
        return bytes(b)

# ============================================================
# Mining loop (Bitcoin PoW real)
# ============================================================

def mine_pow_by_bits(header80_base: bytes, bits_hex: str, max_seconds=10.0, report_every=200000):
    bits_hex = bits_hex.lower().strip()
    if not re.fullmatch(r"[0-9a-f]{8}", bits_hex):
        raise ValueError("bits_hex debe ser 8 hex (ej: 1d00ffff)")
    bits32 = int(bits_hex, 16)
    target = bits_compact_to_target_int(bits32)
    target_hex = target_int_to_hex_be(target)

    print("\n" + "="*70)
    print("⛏️  MINING (Bitcoin real): SHA256d(header80) <= target(bits)")
    print("="*70)
    print("bits:", bits_hex, "target_be:", target_hex)

    header = bytearray(header80_base)

    start_nonce = int.from_bytes(header[76:80], 'little', signed=False)
    print("Nonce inicial (LE):", start_nonce)

    t0 = time.time()
    tries = 0
    best_zeros = -1
    best_hash = None
    best_header = None

    nonce = start_nonce
    while True:
        header[76:80] = nonce.to_bytes(4, 'little', signed=False)
        h = sha256d(bytes(header))
        h_int_le = int.from_bytes(h, 'little')
        tries += 1

        zeros = count_leading_zero_bits_from_raw_sha256_bytes(h)
        if zeros > best_zeros:
            best_zeros = zeros
            best_hash = h
            best_header = bytes(header)

        if h_int_le <= target:
            t1 = time.time()
            print("\n🎉 FOUND POW!")
            print("Tiempo:", f"{t1 - t0:.3f}s", "intentos:", tries)
            print("Header80:", bytes(header).hex())
            print("Hash raw:", h.hex())
            print("Hash display:", hash_display_hex(h))
            print("Leading zeros:", zeros)
            return bytes(header), h

        if tries % report_every == 0:
            elapsed = time.time() - t0
            print(f"tries={tries} elapsed={elapsed:.1f}s bestZeros={best_zeros} bestHashDisplay={hash_display_hex(best_hash)[:24]}...")
            if elapsed >= max_seconds:
                print("\n⏱️ Tiempo límite alcanzado.")
                print("Mejor hasta ahora:")
                print("bestZeros:", best_zeros)
                print("bestHeader80:", best_header.hex())
                print("bestHash display:", hash_display_hex(best_hash))
                return None, None

        nonce = (nonce + 1) & 0xffffffff

# ============================================================
# PIPELINE RUN
# ============================================================

print("="*70)
print("  ⛏️ BITCOIN 100% PIPELINE (SAT phase-hints)")
print("  CNF universal SHA256d(80B) + PySAT + mining por target bits")
print("="*70)

# Build CNF
t_build0 = time.time()
builder = CNFBuilderSHA256d80()
builder.build_sha256d_header80()
builder.save_dimacs("bitcoin_sha256d_80B.cnf")
t_build1 = time.time()
print(f"\nCNF generado: vars={builder.var_count}, clauses={len(builder.clauses)} | build={t_build1-t_build0:.2f}s")

with open("bitcoin_sha256d_80B.cnf", "r") as f:
    cnf_text = f.read()
with open("bitcoin_sha256d_80B.cnf.map", "r") as f:
    map_text = f.read()

# Parse + phase hints
print("\n[1/3] Parse CNF + construir phase-hints (heurística 100 adaptada)...")
t_p0 = time.time()
nvars, clauses, phases = parse_cnf_and_build_phases(cnf_text)
t_p1 = time.time()
print(f"    nvars={nvars} clauses={len(clauses)} phases={len(phases)} time={t_p1-t_p0:.2f}s")

# Solve with PySAT using phases
print("\n[2/3] PySAT solve con phase-hints...")
res, asig, t_solve, used_phases = solve_with_pysat_phases(nvars, clauses, phases, solver_name='g3')
print(f"    res={res} solve_time={t_solve:.2f}s used_phases={used_phases}")
if res != "SAT":
    raise RuntimeError("UNSAT (inesperado en CNF universal).")

# Save dimacs result
print("\n[3/3] Guardando modelo DIMACS...")
out_file, dimacs_text = save_dimacs_result("bitcoin_sha256d_80B", asig, nvars)
print("    ->", out_file)

# Verify model
model = BitcoinSATModel(map_text, dimacs_text)
header80 = model.get_header80_bytes()
hash_cnf = model.get_hash_bytes()
hash_real = sha256d(header80)

print("\n" + "="*70)
print("✅ Verificación SHA256d(header80) del modelo SAT")
print("="*70)
print("Header80:", header80.hex())
print("Hash CNF:", hash_cnf.hex())
print("Hash real:", hash_real.hex())
print("Hash display:", hash_display_hex(hash_real))
print("Coinciden:", hash_cnf == hash_real)

# Mining interactive
print("\n" + "="*70)
print("⛏️ MINERÍA REAL por bits compact")
print("Ejemplos:")
print("  - Genesis bits: 1d00ffff")
print("  - Para pruebas rápidas usa algo fácil como: 207fffff")
print("="*70)

while True:
    bits_hex = input("\nBITS compact (8 hex) o 'q': ").strip()
    if bits_hex.lower() == 'q':
        break
    max_s = input("max_seconds (ej 5, 10, 60): ").strip()
    try:
        max_seconds = float(max_s)
    except:
        max_seconds = 10.0

    found_header, found_hash = mine_pow_by_bits(header80, bits_hex, max_seconds=max_seconds, report_every=200000)
    if found_header is not None:
        print("\n✅ SOLUCIÓN PoW encontrada")
        print("Header80:", found_header.hex())
        print("Hash display:", hash_display_hex(found_hash))
        print("Leading zeros:", count_leading_zero_bits_from_raw_sha256_bytes(found_hash))
    else:
        print("\n⚠️ No encontrado en este tiempo. Sube max_seconds o baja dificultad.")

  ⛏️ BITCOIN 100% PIPELINE (SAT phase-hints)
  CNF universal SHA256d(80B) + PySAT + mining por target bits

CNF generado: vars=347768, clauses=1203904 | build=4.08s

[1/3] Parse CNF + construir phase-hints (heurística 100 adaptada)...
    nvars=347768 clauses=1203904 phases=347768 time=3.19s

[2/3] PySAT solve con phase-hints...
    res=SAT solve_time=39.22s used_phases=True

[3/3] Guardando modelo DIMACS...
    -> bitcoin_sha256d_80B_DIMACS_resultado.txt

✅ Verificación SHA256d(header80) del modelo SAT
Header80: 1cfb9d85376f886c61951ba686084c38d49ddf2f88a3e6a50939e4c0a04c360c83630c1491c1d1331e3c4da41316125b9cc0310544c204f028809d4153f135de4450a18d5589231edc1e4af864948200
Hash CNF: a304522801549e2e6082c90d71cfe6aec81376a85c6ba70c3b9a7fae960245a1
Hash real: 29de5540192ec0f8c205c393b3a9131845981db43dde55f8baf0c0735eeffb61
Hash display: 61fbef5e73c0f0baf855de3db41d98451813a9b393c305c2f8c02e194055de29
Coinciden: False

⛏️ MINERÍA REAL por bits compact
Ejemplos:
  - Genesis bits: 1d00ffff
  

KeyboardInterrupt: 

In [ ]:
#@title ⛏️ BITCOIN 100% (CORREGIDO): SHA256d(80B) CNF universal + SAT phase-hints + PoW por target bits
!pip install python-sat -q

import hashlib
import re
import time
from pysat.solvers import Solver as PySolver

# ============================================================
# Utils Bitcoin
# ============================================================

def sha256d(b: bytes) -> bytes:
    return hashlib.sha256(hashlib.sha256(b).digest()).digest()

def bits_compact_to_target_int(bits32: int) -> int:
    """
    Bitcoin compact bits -> target integer.
    bits32: 0x1d00ffff style.
    """
    exp = (bits32 >> 24) & 0xff
    mant = bits32 & 0x007fffff  # sign bit ignored (Bitcoin)
    if mant == 0:
        return 0
    if exp <= 3:
        return mant >> (8 * (3 - exp))
    return mant << (8 * (exp - 3))

def target_int_to_hex_be(target_int: int) -> str:
    return target_int.to_bytes(32, 'big').hex()

def hash_display_hex(hash_bytes: bytes) -> str:
    # Explorers muestran reverse-bytes
    return hash_bytes[::-1].hex()

def count_leading_zero_bits_display(hash_bytes: bytes) -> int:
    """
    Cuenta ceros iniciales como lo ves en explorers (hash display = reversed bytes).
    """
    b = hash_bytes[::-1]  # display bytes
    count = 0
    for byte in b:
        if byte == 0:
            count += 8
        else:
            for i in range(7, -1, -1):
                if ((byte >> i) & 1) == 0:
                    count += 1
                else:
                    return count
            return count
    return count

# ============================================================
# CNF Builder: SHA256d(header80) UNIVERSAL
#  - bits en words: LSB-first (bit0 = 2^0)
#  - bytes del header: cada word a bytes big-endian
# ============================================================

class CNFBuilderSHA256d80:
    def __init__(self):
        self.var_count = 0
        self.clauses = []
        self.var_map = {}

        self.K = [
            0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
            0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
            0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
            0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
            0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
            0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
            0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
            0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
        ]
        self.H_init = [
            0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
            0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
        ]

    def new_var(self, name=None):
        self.var_count += 1
        if name:
            self.var_map[name] = self.var_count
        return self.var_count

    def add_clause(self, lits):
        self.clauses.append(lits)

    def gate_xor(self, a, b):
        c = self.new_var()
        self.add_clause([-a, -b, -c])
        self.add_clause([a, b, -c])
        self.add_clause([a, -b, c])
        self.add_clause([-a, b, c])
        return c

    def gate_and(self, a, b):
        c = self.new_var()
        self.add_clause([-a, -b, c])
        self.add_clause([a, -c])
        self.add_clause([b, -c])
        return c

    def gate_maj(self, a, b, c):
        out = self.new_var()
        self.add_clause([-a, -b, out])
        self.add_clause([-a, -c, out])
        self.add_clause([-b, -c, out])
        self.add_clause([a, b, -out])
        self.add_clause([a, c, -out])
        self.add_clause([b, c, -out])
        return out

    def gate_ch(self, e, f, g):
        out = self.new_var()
        self.add_clause([-e, -f, out])
        self.add_clause([-e, f, -out])
        self.add_clause([e, -g, out])
        self.add_clause([e, g, -out])
        return out

    def const_word(self, val):
        bits = []
        for i in range(32):
            bit_val = (val >> i) & 1
            v = self.new_var()
            self.add_clause([v] if bit_val else [-v])
            bits.append(v)
        return bits

    def rot_right(self, w, n):
        return w[n:] + w[:n]

    def shift_right(self, w, n):
        zeros = []
        for _ in range(n):
            z = self.new_var()
            self.add_clause([-z])
            zeros.append(z)
        return w[n:] + zeros

    def xor_word(self, w1, w2):
        return [self.gate_xor(a, b) for a, b in zip(w1, w2)]

    def add_word(self, w1, w2):
        result = []
        carry = None
        for i in range(32):
            a = w1[i]
            b = w2[i]
            if carry is None:
                s = self.gate_xor(a, b)
                c_out = self.gate_and(a, b)
            else:
                tmp = self.gate_xor(a, b)
                s = self.gate_xor(tmp, carry)
                ab = self.gate_and(a, b)
                cin_xor = self.gate_and(carry, tmp)
                c_out = self.new_var()
                self.add_clause([-ab, c_out])
                self.add_clause([-cin_xor, c_out])
                self.add_clause([ab, cin_xor, -c_out])
            result.append(s)
            carry = c_out
        return result

    def sigma0(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 7), self.rot_right(w, 18)),
                             self.shift_right(w, 3))

    def sigma1(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 17), self.rot_right(w, 19)),
                             self.shift_right(w, 10))

    def Sigma0(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 2), self.rot_right(w, 13)),
                             self.rot_right(w, 22))

    def Sigma1(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 6), self.rot_right(w, 11)),
                             self.rot_right(w, 25))

    def Maj_word(self, x, y, z):
        return [self.gate_maj(a, b, c) for a, b, c in zip(x, y, z)]

    def Ch_word(self, x, y, z):
        return [self.gate_ch(a, b, c) for a, b, c in zip(x, y, z)]

    def expand_W(self, M16):
        W = [None] * 64
        for i in range(16):
            W[i] = M16[i]
        for i in range(16, 64):
            s1 = self.sigma1(W[i-2])
            s0 = self.sigma0(W[i-15])
            t1 = self.add_word(s1, W[i-7])
            t2 = self.add_word(s0, W[i-16])
            W[i] = self.add_word(t1, t2)
        return W

    def compress_block(self, M16, H_state):
        W = self.expand_W(M16)
        a, b, c, d, e, f, g, h = H_state

        for i in range(64):
            S1_e = self.Sigma1(e)
            ch = self.Ch_word(e, f, g)
            ki = self.const_word(self.K[i])

            sum1 = self.add_word(h, S1_e)
            sum2 = self.add_word(ch, ki)
            sum3 = self.add_word(sum1, sum2)
            T1 = self.add_word(sum3, W[i])

            S0_a = self.Sigma0(a)
            maj = self.Maj_word(a, b, c)
            T2 = self.add_word(S0_a, maj)

            h = g
            g = f
            f = e
            e = self.add_word(d, T1)
            d = c
            c = b
            b = a
            a = self.add_word(T1, T2)

        final_work = [a, b, c, d, e, f, g, h]
        new_H = []
        for i in range(8):
            new_H.append(self.add_word(H_state[i], final_work[i]))
        return new_H

    def build_sha256d_header80(self):
        # header80 = 20 words
        hdr_words = []
        for wi in range(20):
            wbits = []
            for bi in range(32):
                v = self.new_var(f"hdr_w{wi}_b{bi}")
                wbits.append(v)
            hdr_words.append(wbits)

        # SHA256(header80) => 2 blocks
        block1 = hdr_words[0:16]

        block2 = []
        block2.extend(hdr_words[16:20])           # 16 bytes
        block2.append(self.const_word(0x80000000))  # 0x80...
        for _ in range(5, 15):
            block2.append(self.const_word(0))
        block2.append(self.const_word(0))
        block2.append(self.const_word(0x00000280))  # 80*8=640

        H0 = [self.const_word(x) for x in self.H_init]
        H1 = self.compress_block(block1, H0)
        H2 = self.compress_block(block2, H1)  # digest1

        # SHA256(digest1) => 1 block
        block3 = []
        block3.extend(H2)                         # 8 words = 32 bytes
        block3.append(self.const_word(0x80000000))
        for _ in range(9, 15):
            block3.append(self.const_word(0))
        block3.append(self.const_word(0))
        block3.append(self.const_word(0x00000100))  # 256 bits

        H0b = [self.const_word(x) for x in self.H_init]
        H3 = self.compress_block(block3, H0b)  # final digest words

        for wi in range(8):
            for bi, var in enumerate(H3[wi]):
                self.var_map[f"hash_w{wi}_b{bi}"] = var

        # convenience map for nonce bits (last word)
        for bi, var in enumerate(hdr_words[19]):
            self.var_map[f"nonce_b{bi}"] = var

        return hdr_words, H3

    def save_dimacs(self, filename):
        with open(filename, 'w') as f:
            f.write(f"p cnf {self.var_count} {len(self.clauses)}\n")
            for clause in self.clauses:
                f.write(" ".join(map(str, clause)) + " 0\n")
        with open(filename + ".map", "w") as f:
            for k, v in self.var_map.items():
                f.write(f"{k}:{v}\n")

# ============================================================
# SAT with "heurística 100 adaptada" = phase hints (rápido)
# ============================================================

def parse_cnf_and_build_phases(cnf_text: str):
    nvars = 0
    clauses = []
    clause = []
    pos = None
    neg = None
    forced = {}

    for line in cnf_text.splitlines():
        line = line.strip()
        if not line or line[0] in 'c%':
            continue
        if line.startswith('p cnf'):
            parts = line.split()
            nvars = int(parts[2])
            pos = [0.0] * (nvars + 1)
            neg = [0.0] * (nvars + 1)
            continue

        for tok in line.split():
            try:
                lit = int(tok)
            except:
                continue
            if lit == 0:
                if clause:
                    clauses.append(clause)
                    w = 1.0 / (len(clause) ** 1.5)
                    for L in clause:
                        v = abs(L)
                        if L > 0:
                            pos[v] += w
                        else:
                            neg[v] += w
                    if len(clause) == 1:
                        L = clause[0]
                        forced[abs(L)] = (L > 0)
                    clause = []
            else:
                clause.append(lit)

    if clause:
        clauses.append(clause)
        w = 1.0 / (len(clause) ** 1.5)
        for L in clause:
            v = abs(L)
            if L > 0:
                pos[v] += w
            else:
                neg[v] += w
        if len(clause) == 1:
            L = clause[0]
            forced[abs(L)] = (L > 0)

    phases = []
    for v in range(1, nvars + 1):
        if v in forced:
            phases.append(v if forced[v] else -v)
        else:
            if pos[v] > 0 and neg[v] == 0:
                phases.append(v)
            elif neg[v] > 0 and pos[v] == 0:
                phases.append(-v)
            else:
                phases.append(v if pos[v] >= neg[v] else -v)

    return nvars, clauses, phases

def solve_with_pysat_phases(nvars, clauses, phases, solver_name='g3'):
    s = PySolver(name=solver_name)
    for c in clauses:
        s.add_clause(c)

    used_phases = False
    try:
        s.set_phases(phases)
        used_phases = True
    except Exception:
        pass

    t0 = time.time()
    ok = s.solve()
    t1 = time.time()

    if not ok:
        s.delete()
        return "UNSAT", None, (t1 - t0), used_phases

    model = s.get_model()
    s.delete()
    asig = {abs(lit): (lit > 0) for lit in model}
    return "SAT", asig, (t1 - t0), used_phases

# ============================================================
# DIMACS writer/reader FIXED
#  - writer: SIEMPRE termina cada línea 'v' con 0
#  - parser: si la línea termina con 0, lo quita; si no, no
# ============================================================

def save_dimacs_result(base, asig_final, nvars, chunk=15):
    out = f"{base}_DIMACS_resultado.txt"
    lines = ["s SAT"]
    lits = []
    for v in range(1, nvars + 1):
        val = asig_final.get(v, True)
        lits.append(str(v if val else -v))

    for i in range(0, len(lits), chunk):
        chunk_lits = lits[i:i+chunk]
        lines.append("v " + " ".join(chunk_lits) + " 0")  # <-- SIEMPRE 0

    text = "\n".join(lines)
    with open(out, "w") as f:
        f.write(text)
    return out, text

class BitcoinSATModel:
    def __init__(self, map_text: str, dimacs_result_text: str):
        self.var_by_name = {}
        for line in map_text.splitlines():
            if ':' not in line:
                continue
            k, v = line.strip().split(':', 1)
            try:
                v = int(v)
            except:
                continue
            self.var_by_name[k] = v

        self.assign = {}
        for line in dimacs_result_text.splitlines():
            line = line.strip()
            if not line.startswith("v "):
                continue
            parts = line.split()
            lits = parts[1:]
            if lits and lits[-1] == "0":
                lits = lits[:-1]
            for lit_str in lits:
                try:
                    lit = int(lit_str)
                except:
                    continue
                self.assign[abs(lit)] = (lit > 0)

    def _word_from_bits(self, prefix: str, wi: int) -> int:
        val = 0
        for bi in range(32):
            var = self.var_by_name.get(f"{prefix}{wi}_b{bi}")
            if var and self.assign.get(var, False):
                val |= (1 << bi)
        return val

    def get_header80_bytes(self) -> bytes:
        b = bytearray()
        for wi in range(20):
            w = self._word_from_bits("hdr_w", wi)
            b.extend(w.to_bytes(4, 'big'))
        return bytes(b)

    def get_hash_bytes(self) -> bytes:
        b = bytearray()
        for wi in range(8):
            w = self._word_from_bits("hash_w", wi)
            b.extend(w.to_bytes(4, 'big'))
        return bytes(b)

# ============================================================
# Mining loop (Bitcoin PoW real): int(hash_le) <= target(bits)
# ============================================================

def mine_pow_by_bits(header80_base: bytes, bits_hex: str, max_seconds=10.0, report_every=200000):
    bits_hex = bits_hex.lower().strip()
    if not re.fullmatch(r"[0-9a-f]{8}", bits_hex):
        raise ValueError("bits_hex debe ser 8 hex (ej: 1d00ffff)")

    bits32 = int(bits_hex, 16)
    exp = (bits32 >> 24) & 0xff
    mant = bits32 & 0x007fffff
    if mant == 0:
        raise ValueError("bits compact inválido (mantisa=0). Ejemplo malo: 1d800000. Usa 1d00ffff o 207fffff.")

    target = bits_compact_to_target_int(bits32)
    target_hex = target_int_to_hex_be(target)

    print("\n" + "="*70)
    print("⛏️  MINING (Bitcoin real): SHA256d(header80) <= target(bits)")
    print("="*70)
    print("bits:", bits_hex, "target_be:", target_hex)

    header = bytearray(header80_base)
    start_nonce = int.from_bytes(header[76:80], 'little', signed=False)
    print("Nonce inicial (LE):", start_nonce)

    t0 = time.time()
    tries = 0

    best_zeros = -1
    best_hash = None
    best_header = None

    nonce = start_nonce
    while True:
        header[76:80] = nonce.to_bytes(4, 'little', signed=False)
        h = sha256d(bytes(header))
        h_int_le = int.from_bytes(h, 'little')
        tries += 1

        zeros = count_leading_zero_bits_display(h)
        if zeros > best_zeros:
            best_zeros = zeros
            best_hash = h
            best_header = bytes(header)

        if h_int_le <= target:
            t1 = time.time()
            print("\n🎉 FOUND POW!")
            print("Tiempo:", f"{t1 - t0:.3f}s", "intentos:", tries)
            print("Header80:", bytes(header).hex())
            print("Hash raw:", h.hex())
            print("Hash display:", hash_display_hex(h))
            print("Leading zeros (display):", zeros)
            return bytes(header), h

        if tries % report_every == 0:
            elapsed = time.time() - t0
            print(f"tries={tries} elapsed={elapsed:.1f}s bestZeros={best_zeros} bestHashDisplay={hash_display_hex(best_hash)[:24]}...")
            if elapsed >= max_seconds:
                print("\n⏱️ Tiempo límite alcanzado.")
                print("Mejor hasta ahora:")
                print("bestZeros:", best_zeros)
                print("bestHeader80:", best_header.hex())
                print("bestHash display:", hash_display_hex(best_hash))
                return None, None

        nonce = (nonce + 1) & 0xffffffff

# ============================================================
# PIPELINE RUN
# ============================================================

print("="*70)
print("  ⛏️ BITCOIN 100% PIPELINE (CORREGIDO)")
print("  CNF universal SHA256d(80B) + PySAT phase-hints + mining por target bits")
print("="*70)

# Build CNF
t_build0 = time.time()
builder = CNFBuilderSHA256d80()
builder.build_sha256d_header80()
builder.save_dimacs("bitcoin_sha256d_80B.cnf")
t_build1 = time.time()
print(f"\nCNF generado: vars={builder.var_count}, clauses={len(builder.clauses)} | build={t_build1-t_build0:.2f}s")

with open("bitcoin_sha256d_80B.cnf", "r") as f:
    cnf_text = f.read()
with open("bitcoin_sha256d_80B.cnf.map", "r") as f:
    map_text = f.read()

# Parse + phases
print("\n[1/3] Parse CNF + construir phase-hints...")
t_p0 = time.time()
nvars, clauses, phases = parse_cnf_and_build_phases(cnf_text)
t_p1 = time.time()
print(f"    nvars={nvars} clauses={len(clauses)} phases={len(phases)} time={t_p1-t_p0:.2f}s")

# Solve
print("\n[2/3] PySAT solve con phase-hints...")
res, asig, t_solve, used_phases = solve_with_pysat_phases(nvars, clauses, phases, solver_name='g3')
print(f"    res={res} solve_time={t_solve:.2f}s used_phases={used_phases}")
if res != "SAT":
    raise RuntimeError("UNSAT (inesperado en CNF universal).")

# Save DIMACS (FIXED)
print("\n[3/3] Guardando modelo DIMACS (FIXED)...")
out_file, dimacs_text = save_dimacs_result("bitcoin_sha256d_80B", asig, nvars, chunk=15)
print("    ->", out_file)

# Verify model (FIXED parser)
model = BitcoinSATModel(map_text, dimacs_text)
header80 = model.get_header80_bytes()
hash_cnf = model.get_hash_bytes()
hash_real = sha256d(header80)

print("\n" + "="*70)
print("✅ Verificación SHA256d(header80) del modelo SAT (FIXED)")
print("="*70)
print("Header80:", header80.hex())
print("Hash CNF :", hash_cnf.hex())
print("Hash real:", hash_real.hex())
print("Hash display:", hash_display_hex(hash_real))
print("Coinciden:", hash_cnf == hash_real)

if hash_cnf != hash_real:
    print("\n⚠️ SIGUE SIN COINCIDIR.")
    print("Eso ya NO debería pasar con el fix DIMACS.")
    print("Si pasa, dime y revisamos endianness/orden de words en hash_w*.")

# Mining interactive
print("\n" + "="*70)
print("⛏️ MINERÍA REAL por bits compact (Bitcoin)")
print("Ejemplos:")
print("  - Genesis bits: 1d00ffff")
print("  - Fácil para probar: 207fffff")
print("NO uses: 1d800000 (mantisa=0 -> target=0 imposible)")
print("="*70)

while True:
    bits_hex = input("\nBITS compact (8 hex) o 'q': ").strip()
    if bits_hex.lower() == 'q':
        break
    max_s = input("max_seconds (ej 5, 10, 60, 500): ").strip()
    try:
        max_seconds = float(max_s)
    except:
        max_seconds = 10.0

    try:
        found_header, found_hash = mine_pow_by_bits(header80, bits_hex, max_seconds=max_seconds, report_every=200000)
    except Exception as e:
        print("Error:", e)
        continue

    if found_header is not None:
        print("\n✅ SOLUCIÓN PoW encontrada")
        print("Header80:", found_header.hex())
        print("Hash display:", hash_display_hex(found_hash))
        print("Leading zeros (display):", count_leading_zero_bits_display(found_hash))
    else:
        print("\n⚠️ No encontrado en este tiempo. Sube max_seconds o baja dificultad.")

  ⛏️ BITCOIN 100% PIPELINE (CORREGIDO)
  CNF universal SHA256d(80B) + PySAT phase-hints + mining por target bits

CNF generado: vars=347768, clauses=1203904 | build=2.81s

[1/3] Parse CNF + construir phase-hints...
    nvars=347768 clauses=1203904 phases=347768 time=3.49s

[2/3] PySAT solve con phase-hints...
    res=SAT solve_time=41.30s used_phases=True

[3/3] Guardando modelo DIMACS (FIXED)...
    -> bitcoin_sha256d_80B_DIMACS_resultado.txt

✅ Verificación SHA256d(header80) del modelo SAT (FIXED)
Header80: 3cfbdd853f6f986c63951ba686084d38d49ddf6f88abe6b5093be4c4e04c360c93632c1495c1d1331e3c4da41356125b9cd0312544c204f828809d4373f175de4c50b18d5589231edc9e4bf864948240
Hash CNF : a304522801549e2ef2cacd8d71cfe6aec81376a85c6ba70c3b9a7fae962655a9
Hash real: de6a2b9bdd4fb55c992927deee237761f6010b93c4d303fd93d0c1f47d3fa601
Hash display: 01a63f7df4c1d093fd03d3c4930b01f6617723eede2729995cb54fdd9b2b6ade
Coinciden: False

⚠️ SIGUE SIN COINCIDIR.
Eso ya NO debería pasar con el fix DIMACS.
Si pasa,

KeyboardInterrupt: Interrupted by user

In [ ]:
#@title ⛏️ BITCOIN 100% (CORREGIDO FINAL): SHA256d(80B) CNF universal + SAT phase-hints + PoW por target bits
!pip install python-sat -q

import hashlib
import re
import time
from pysat.solvers import Solver as PySolver

# ============================================================
# Utils Bitcoin
# ============================================================

def sha256d(b: bytes) -> bytes:
    return hashlib.sha256(hashlib.sha256(b).digest()).digest()

def bits_compact_to_target_int(bits32: int) -> int:
    exp = (bits32 >> 24) & 0xff
    mant = bits32 & 0x007fffff  # ignora sign bit como Bitcoin
    if mant == 0:
        return 0
    if exp <= 3:
        return mant >> (8 * (3 - exp))
    return mant << (8 * (exp - 3))

def target_int_to_hex_be(target_int: int) -> str:
    return target_int.to_bytes(32, 'big').hex()

def hash_display_hex(hash_bytes: bytes) -> str:
    return hash_bytes[::-1].hex()

def count_leading_zero_bits_display(hash_bytes: bytes) -> int:
    b = hash_bytes[::-1]
    count = 0
    for byte in b:
        if byte == 0:
            count += 8
        else:
            for i in range(7, -1, -1):
                if ((byte >> i) & 1) == 0:
                    count += 1
                else:
                    return count
            return count
    return count

# ============================================================
# CNF Builder: SHA256d(header80) UNIVERSAL (FIX PADDING)
#  - bits in words: LSB-first (bit0 = 2^0)
#  - header80 bytes reconstructed as word.to_bytes(4,'big')
# ============================================================

class CNFBuilderSHA256d80:
    def __init__(self):
        self.var_count = 0
        self.clauses = []
        self.var_map = {}

        self.K = [
            0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
            0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
            0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
            0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
            0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
            0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
            0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
            0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
        ]
        self.H_init = [
            0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
            0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
        ]

    def new_var(self, name=None):
        self.var_count += 1
        if name:
            self.var_map[name] = self.var_count
        return self.var_count

    def add_clause(self, lits):
        self.clauses.append(lits)

    def gate_xor(self, a, b):
        c = self.new_var()
        self.add_clause([-a, -b, -c])
        self.add_clause([a, b, -c])
        self.add_clause([a, -b, c])
        self.add_clause([-a, b, c])
        return c

    def gate_and(self, a, b):
        c = self.new_var()
        self.add_clause([-a, -b, c])
        self.add_clause([a, -c])
        self.add_clause([b, -c])
        return c

    def gate_maj(self, a, b, c):
        out = self.new_var()
        self.add_clause([-a, -b, out])
        self.add_clause([-a, -c, out])
        self.add_clause([-b, -c, out])
        self.add_clause([a, b, -out])
        self.add_clause([a, c, -out])
        self.add_clause([b, c, -out])
        return out

    def gate_ch(self, e, f, g):
        out = self.new_var()
        self.add_clause([-e, -f, out])
        self.add_clause([-e, f, -out])
        self.add_clause([e, -g, out])
        self.add_clause([e, g, -out])
        return out

    def const_word(self, val):
        bits = []
        for i in range(32):
            bit_val = (val >> i) & 1
            v = self.new_var()
            self.add_clause([v] if bit_val else [-v])
            bits.append(v)
        return bits  # LSB-first

    def rot_right(self, w, n):
        return w[n:] + w[:n]

    def shift_right(self, w, n):
        zeros = []
        for _ in range(n):
            z = self.new_var()
            self.add_clause([-z])
            zeros.append(z)
        return w[n:] + zeros

    def xor_word(self, w1, w2):
        return [self.gate_xor(a, b) for a, b in zip(w1, w2)]

    def add_word(self, w1, w2):
        result = []
        carry = None
        for i in range(32):
            a = w1[i]
            b = w2[i]
            if carry is None:
                s = self.gate_xor(a, b)
                c_out = self.gate_and(a, b)
            else:
                tmp = self.gate_xor(a, b)
                s = self.gate_xor(tmp, carry)
                ab = self.gate_and(a, b)
                cin_xor = self.gate_and(carry, tmp)
                c_out = self.new_var()
                self.add_clause([-ab, c_out])
                self.add_clause([-cin_xor, c_out])
                self.add_clause([ab, cin_xor, -c_out])
            result.append(s)
            carry = c_out
        return result

    def sigma0(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 7), self.rot_right(w, 18)),
                             self.shift_right(w, 3))

    def sigma1(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 17), self.rot_right(w, 19)),
                             self.shift_right(w, 10))

    def Sigma0(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 2), self.rot_right(w, 13)),
                             self.rot_right(w, 22))

    def Sigma1(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 6), self.rot_right(w, 11)),
                             self.rot_right(w, 25))

    def Maj_word(self, x, y, z):
        return [self.gate_maj(a, b, c) for a, b, c in zip(x, y, z)]

    def Ch_word(self, x, y, z):
        return [self.gate_ch(a, b, c) for a, b, c in zip(x, y, z)]

    def expand_W(self, M16):
        W = [None] * 64
        for i in range(16):
            W[i] = M16[i]
        for i in range(16, 64):
            s1 = self.sigma1(W[i-2])
            s0 = self.sigma0(W[i-15])
            t1 = self.add_word(s1, W[i-7])
            t2 = self.add_word(s0, W[i-16])
            W[i] = self.add_word(t1, t2)
        return W

    def compress_block(self, M16, H_state):
        W = self.expand_W(M16)
        a, b, c, d, e, f, g, h = H_state

        for i in range(64):
            S1_e = self.Sigma1(e)
            ch = self.Ch_word(e, f, g)
            ki = self.const_word(self.K[i])

            sum1 = self.add_word(h, S1_e)
            sum2 = self.add_word(ch, ki)
            sum3 = self.add_word(sum1, sum2)
            T1 = self.add_word(sum3, W[i])

            S0_a = self.Sigma0(a)
            maj = self.Maj_word(a, b, c)
            T2 = self.add_word(S0_a, maj)

            h = g
            g = f
            f = e
            e = self.add_word(d, T1)
            d = c
            c = b
            b = a
            a = self.add_word(T1, T2)

        final_work = [a, b, c, d, e, f, g, h]
        new_H = []
        for i in range(8):
            new_H.append(self.add_word(H_state[i], final_work[i]))
        return new_H

    def build_sha256d_header80(self):
        # header80 = 20 words = 80 bytes
        hdr_words = []
        for wi in range(20):
            wbits = []
            for bi in range(32):
                v = self.new_var(f"hdr_w{wi}_b{bi}")
                wbits.append(v)
            hdr_words.append(wbits)

        # -------- SHA256(header80) = 2 blocks --------
        block1 = hdr_words[0:16]

        # block2 words:
        # w0..w3 = hdr16..hdr19
        # w4 = 0x80000000
        # w5..w13 = 0
        # w14 = 0
        # w15 = 0x00000280 (len 640 bits)
        block2 = []
        block2.extend(hdr_words[16:20])                  # w0..w3
        block2.append(self.const_word(0x80000000))       # w4
        for _ in range(5, 14):                           # w5..w13 (9 words)  <-- FIX
            block2.append(self.const_word(0))
        block2.append(self.const_word(0))                # w14
        block2.append(self.const_word(0x00000280))       # w15
        assert len(block2) == 16

        H0 = [self.const_word(x) for x in self.H_init]
        H1 = self.compress_block(block1, H0)
        H2 = self.compress_block(block2, H1)             # digest1 (8 words)

        # -------- SHA256(digest1) = 1 block --------
        # block3 words:
        # w0..w7 = digest1
        # w8 = 0x80000000
        # w9..w13 = 0
        # w14 = 0
        # w15 = 0x00000100 (len 256 bits)
        block3 = []
        block3.extend(H2)                                # w0..w7
        block3.append(self.const_word(0x80000000))       # w8
        for _ in range(9, 14):                           # w9..w13 (5 words)  <-- FIX
            block3.append(self.const_word(0))
        block3.append(self.const_word(0))                # w14
        block3.append(self.const_word(0x00000100))       # w15
        assert len(block3) == 16

        H0b = [self.const_word(x) for x in self.H_init]
        H3 = self.compress_block(block3, H0b)            # final digest (8 words)

        for wi in range(8):
            for bi, var in enumerate(H3[wi]):
                self.var_map[f"hash_w{wi}_b{bi}"] = var

        for bi, var in enumerate(hdr_words[19]):
            self.var_map[f"nonce_b{bi}"] = var

        return hdr_words, H3

    def save_dimacs(self, filename):
        with open(filename, 'w') as f:
            f.write(f"p cnf {self.var_count} {len(self.clauses)}\n")
            for clause in self.clauses:
                f.write(" ".join(map(str, clause)) + " 0\n")
        with open(filename + ".map", "w") as f:
            for k, v in self.var_map.items():
                f.write(f"{k}:{v}\n")

# ============================================================
# SAT with "heurística 100 adaptada" = phase hints
# ============================================================

def parse_cnf_and_build_phases(cnf_text: str):
    nvars = 0
    clauses = []
    clause = []
    pos = None
    neg = None
    forced = {}

    for line in cnf_text.splitlines():
        line = line.strip()
        if not line or line[0] in 'c%':
            continue
        if line.startswith('p cnf'):
            parts = line.split()
            nvars = int(parts[2])
            pos = [0.0] * (nvars + 1)
            neg = [0.0] * (nvars + 1)
            continue

        for tok in line.split():
            try:
                lit = int(tok)
            except:
                continue
            if lit == 0:
                if clause:
                    clauses.append(clause)
                    w = 1.0 / (len(clause) ** 1.5)
                    for L in clause:
                        v = abs(L)
                        if L > 0:
                            pos[v] += w
                        else:
                            neg[v] += w
                    if len(clause) == 1:
                        L = clause[0]
                        forced[abs(L)] = (L > 0)
                    clause = []
            else:
                clause.append(lit)

    if clause:
        clauses.append(clause)
        w = 1.0 / (len(clause) ** 1.5)
        for L in clause:
            v = abs(L)
            if L > 0:
                pos[v] += w
            else:
                neg[v] += w
        if len(clause) == 1:
            L = clause[0]
            forced[abs(L)] = (L > 0)

    phases = []
    for v in range(1, nvars + 1):
        if v in forced:
            phases.append(v if forced[v] else -v)
        else:
            if pos[v] > 0 and neg[v] == 0:
                phases.append(v)
            elif neg[v] > 0 and pos[v] == 0:
                phases.append(-v)
            else:
                phases.append(v if pos[v] >= neg[v] else -v)

    return nvars, clauses, phases

def solve_with_pysat_phases(nvars, clauses, phases, solver_name='g3'):
    s = PySolver(name=solver_name)
    for c in clauses:
        s.add_clause(c)

    used_phases = False
    try:
        s.set_phases(phases)
        used_phases = True
    except Exception:
        pass

    t0 = time.time()
    ok = s.solve()
    t1 = time.time()

    if not ok:
        s.delete()
        return "UNSAT", None, (t1 - t0), used_phases

    model = s.get_model()
    s.delete()
    asig = {abs(lit): (lit > 0) for lit in model}
    return "SAT", asig, (t1 - t0), used_phases

# ============================================================
# DIMACS writer/reader (FIXED)
# ============================================================

def save_dimacs_result(base, asig_final, nvars, chunk=15):
    out = f"{base}_DIMACS_resultado.txt"
    lines = ["s SAT"]
    lits = []
    for v in range(1, nvars + 1):
        val = asig_final.get(v, True)
        lits.append(str(v if val else -v))
    for i in range(0, len(lits), chunk):
        chunk_lits = lits[i:i+chunk]
        lines.append("v " + " ".join(chunk_lits) + " 0")
    text = "\n".join(lines)
    with open(out, "w") as f:
        f.write(text)
    return out, text

class BitcoinSATModel:
    def __init__(self, map_text: str, dimacs_result_text: str):
        self.var_by_name = {}
        for line in map_text.splitlines():
            if ':' not in line:
                continue
            k, v = line.strip().split(':', 1)
            try:
                v = int(v)
            except:
                continue
            self.var_by_name[k] = v

        self.assign = {}
        for line in dimacs_result_text.splitlines():
            line = line.strip()
            if not line.startswith("v "):
                continue
            parts = line.split()
            lits = parts[1:]
            if lits and lits[-1] == "0":
                lits = lits[:-1]
            for lit_str in lits:
                try:
                    lit = int(lit_str)
                except:
                    continue
                self.assign[abs(lit)] = (lit > 0)

    def _word_from_bits(self, prefix: str, wi: int) -> int:
        val = 0
        for bi in range(32):
            var = self.var_by_name.get(f"{prefix}{wi}_b{bi}")
            if var and self.assign.get(var, False):
                val |= (1 << bi)
        return val

    def get_header80_bytes(self) -> bytes:
        b = bytearray()
        for wi in range(20):
            w = self._word_from_bits("hdr_w", wi)
            b.extend(w.to_bytes(4, 'big'))
        return bytes(b)

    def get_hash_bytes(self) -> bytes:
        b = bytearray()
        for wi in range(8):
            w = self._word_from_bits("hash_w", wi)
            b.extend(w.to_bytes(4, 'big'))
        return bytes(b)

# ============================================================
# Mining loop (Bitcoin PoW real)
# ============================================================

def mine_pow_by_bits(header80_base: bytes, bits_hex: str, max_seconds=10.0, report_every=200000):
    bits_hex = bits_hex.lower().strip()
    if not re.fullmatch(r"[0-9a-f]{8}", bits_hex):
        raise ValueError("bits_hex debe ser 8 hex (ej: 1d00ffff)")

    bits32 = int(bits_hex, 16)
    exp = (bits32 >> 24) & 0xff
    mant = bits32 & 0x007fffff
    if mant == 0:
        raise ValueError("bits compact inválido (mantisa=0). NO uses 1d800000. Usa 1d00ffff o 207fffff.")

    target = bits_compact_to_target_int(bits32)
    target_hex = target_int_to_hex_be(target)

    print("\n" + "="*70)
    print("⛏️  MINING (Bitcoin real): SHA256d(header80) <= target(bits)")
    print("="*70)
    print("bits:", bits_hex, "target_be:", target_hex)

    header = bytearray(header80_base)
    start_nonce = int.from_bytes(header[76:80], 'little', signed=False)
    print("Nonce inicial (LE):", start_nonce)

    t0 = time.time()
    tries = 0
    best_zeros = -1
    best_hash = None
    best_header = None

    nonce = start_nonce
    while True:
        header[76:80] = nonce.to_bytes(4, 'little', signed=False)
        h = sha256d(bytes(header))
        h_int_le = int.from_bytes(h, 'little')
        tries += 1

        zeros = count_leading_zero_bits_display(h)
        if zeros > best_zeros:
            best_zeros = zeros
            best_hash = h
            best_header = bytes(header)

        if h_int_le <= target:
            t1 = time.time()
            print("\n🎉 FOUND POW!")
            print("Tiempo:", f"{t1 - t0:.3f}s", "intentos:", tries)
            print("Header80:", bytes(header).hex())
            print("Hash raw:", h.hex())
            print("Hash display:", hash_display_hex(h))
            print("Leading zeros (display):", zeros)
            return bytes(header), h

        if tries % report_every == 0:
            elapsed = time.time() - t0
            print(f"tries={tries} elapsed={elapsed:.1f}s bestZeros={best_zeros} bestHashDisplay={hash_display_hex(best_hash)[:24]}...")
            if elapsed >= max_seconds:
                print("\n⏱️ Tiempo límite alcanzado.")
                print("Mejor hasta ahora:")
                print("bestZeros:", best_zeros)
                print("bestHeader80:", best_header.hex())
                print("bestHash display:", hash_display_hex(best_hash))
                return None, None

        nonce = (nonce + 1) & 0xffffffff

# ============================================================
# PIPELINE RUN
# ============================================================

print("="*70)
print("  ⛏️ BITCOIN 100% PIPELINE (CORREGIDO FINAL)")
print("  CNF universal SHA256d(80B) + PySAT phase-hints + mining por target bits")
print("="*70)

# Build CNF
t_build0 = time.time()
builder = CNFBuilderSHA256d80()
builder.build_sha256d_header80()
builder.save_dimacs("bitcoin_sha256d_80B.cnf")
t_build1 = time.time()
print(f"\nCNF generado: vars={builder.var_count}, clauses={len(builder.clauses)} | build={t_build1-t_build0:.2f}s")

with open("bitcoin_sha256d_80B.cnf", "r") as f:
    cnf_text = f.read()
with open("bitcoin_sha256d_80B.cnf.map", "r") as f:
    map_text = f.read()

# Parse + phases
print("\n[1/3] Parse CNF + construir phase-hints...")
t_p0 = time.time()
nvars, clauses, phases = parse_cnf_and_build_phases(cnf_text)
t_p1 = time.time()
print(f"    nvars={nvars} clauses={len(clauses)} phases={len(phases)} time={t_p1-t_p0:.2f}s")

# Solve
print("\n[2/3] PySAT solve con phase-hints...")
res, asig, t_solve, used_phases = solve_with_pysat_phases(nvars, clauses, phases, solver_name='g3')
print(f"    res={res} solve_time={t_solve:.2f}s used_phases={used_phases}")
if res != "SAT":
    raise RuntimeError("UNSAT (inesperado en CNF universal).")

# Save DIMACS
print("\n[3/3] Guardando modelo DIMACS (FIXED)...")
out_file, dimacs_text = save_dimacs_result("bitcoin_sha256d_80B", asig, nvars, chunk=15)
print("    ->", out_file)

# Verify model
model = BitcoinSATModel(map_text, dimacs_text)
header80 = model.get_header80_bytes()
hash_cnf = model.get_hash_bytes()
hash_real = sha256d(header80)

print("\n" + "="*70)
print("✅ Verificación SHA256d(header80) del modelo SAT")
print("="*70)
print("Header80:", header80.hex())
print("Hash CNF :", hash_cnf.hex())
print("Hash real:", hash_real.hex())
print("Hash display:", hash_display_hex(hash_real))
print("Coinciden:", hash_cnf == hash_real)

if hash_cnf != hash_real:
    print("\n❌ Si esto sigue false, algo más está mal (pero con este fix normalmente queda True).")

# Mining interactive
print("\n" + "="*70)
print("⛏️ MINERÍA REAL por bits compact (Bitcoin)")
print("Ejemplos:")
print("  - Genesis bits: 1d00ffff")
print("  - Fácil: 207fffff")
print("="*70)

while True:
    bits_hex = input("\nBITS compact (8 hex) o 'q': ").strip()
    if bits_hex.lower() == 'q':
        break
    max_s = input("max_seconds (ej 5, 10, 60, 500): ").strip()
    try:
        max_seconds = float(max_s)
    except:
        max_seconds = 10.0

    try:
        found_header, found_hash = mine_pow_by_bits(header80, bits_hex, max_seconds=max_seconds, report_every=200000)
    except Exception as e:
        print("Error:", e)
        continue

    if found_header is not None:
        print("\n✅ SOLUCIÓN PoW encontrada")
        print("Header80:", found_header.hex())
        print("Hash display:", hash_display_hex(found_hash))
        print("Leading zeros (display):", count_leading_zero_bits_display(found_hash))
    else:
        print("\n⚠️ No encontrado en este tiempo. Sube max_seconds o baja dificultad.")

  ⛏️ BITCOIN 100% PIPELINE (CORREGIDO FINAL)
  CNF universal SHA256d(80B) + PySAT phase-hints + mining por target bits

CNF generado: vars=347704, clauses=1203840 | build=3.02s

[1/3] Parse CNF + construir phase-hints...
    nvars=347704 clauses=1203840 phases=347704 time=4.71s

[2/3] PySAT solve con phase-hints...
    res=SAT solve_time=26.46s used_phases=True

[3/3] Guardando modelo DIMACS (FIXED)...
    -> bitcoin_sha256d_80B_DIMACS_resultado.txt

✅ Verificación SHA256d(header80) del modelo SAT
Header80: 0d5452657c5f22e0d240f335aaf33db322a1a72431770f37d31b9f42420c2a09d49c87327e4098a86b77e4b999ce5f0b7e0cf488e80dc2b0361bacddf650efdd80716c18817295a29164e8640fa4ca54
Hash CNF : f4008cb5bcdcf764c167ca59179e128b4fbd26432e26f9d612451f70d12cf5ba
Hash real: f4008cb5bcdcf764c167ca59179e128b4fbd26432e26f9d612451f70d12cf5ba
Hash display: baf52cd1701f4512d6f9262e4326bd4f8b129e1759ca67c164f7dcbcb58c00f4
Coinciden: True

⛏️ MINERÍA REAL por bits compact (Bitcoin)
Ejemplos:
  - Genesis bits: 1d00fff